<a href="https://colab.research.google.com/github/chltjs1921/2024_modeling_practice/blob/main/Graph_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/atzkenneth/lsfml/tree/master
replicate  
graph 객체부터 우선 만들어보기

In [19]:
import torch
from torch.utils.data import Dataset
#from torchvision import transforms, datasets
import numpy as np

In [20]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')
#출처: https://think-tech.tistory.com/36 [자윰이의 성장일기:티스토리]

In [ ]:
!pip install rdkit
!pip install torch-geometric==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909879 sha256=5647f23cd5aa96cab91dc50ba71b695425d6d98679e9f7e8e868a9d8b9f739ac
  Stored in directory: /root/.cache/pip/wheels/5f/66/21/18ca6ca537cd87a2f01e1c7504922a4bb5a9c2ff9dd36cbe1e
Successfully built torch-geometric


##get_model 구현

In [ ]:
import configparser
import os

import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from torch_geometric.nn import MessagePassing
from torch_geometric.typing import Adj, Size, Tensor

def weights_init(m):
    """Xavier uniform weight initialization.

    :param m: A list of learnable linear PyTorch modules.
    :type m: [torch.nn.modules.linear.Linear]
    """
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

def fourier_encode_dist(x, num_encodings=4, include_self=True):
    """Encoding Euclidian diatomic distances into Fourier features.

    :param x: Distances in Angström.
    :type x: Tensor
    :param num_encodings: Number of sine and cosine functions, defaults to 4
    :type num_encodings: int, optional
    :param include_self: Option to include absolute distance, defaults to True
    :type include_self: bool, optional
    :return: Fourier features.
    :rtype: Tensor
    """
    x = x.unsqueeze(-1)
    device, dtype, orig_x = x.device, x.dtype, x
    scales = 2 ** torch.arange(num_encodings, device=device, dtype=dtype)
    x = x / scales
    x = torch.cat([x.sin(), x.cos()], dim=-1)
    x = torch.cat((x, orig_x), dim=-1) if include_self else x
    return x

In [155]:
#! /usr/bin/env python
# -*- coding: utf-8
#
# Copyright (©) 2023, ETH Zurich

import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from torch_geometric.nn import MessagePassing
from torch_geometric.typing import Adj, Size, Tensor


class DeltaNetAtomic(nn.Module):
    """Atomistic graph neural network (aGNN) for partial charge predictions."""

    def __init__(
        self,
        embedding_dim=128,
        n_kernels=2,
        n_mlp=3,
        mlp_dim=256,
        n_outputs=1,
        m_dim=64,
        initialize_weights=True,
        fourier_features=4,
        aggr="mean",
    ):
        """Initialization of aGNN.

        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions and atomic features), defaults to 64
        :type embeddings_dim: int, optional
        :param n_kernels: Number of message passing functions, defaults to 3
        :type n_kernels: int, optional
        :param n_mlp: Number of multi layer perceptrons, defaults to 3
        :type n_mlp: int, optional
        :param mlp_dim: Feature dimension of multi layer perceptrons, defaults to 256
        :type mlp_dim: int, optional
        :param out_dim: Output dimesion, defaults to 1
        :type out_dim: int, optional
        :param m_dim: Meessage passing feature dimesnion, defaults to 64
        :type m_dim: int, optional
        :param initialize_weights: initialize weights before training, defaults to True
        :type initialize_weights: bool, optional
        :param fourier_features: Number of Fourier features, defaults to 16
        :type fourier_features: int, optional
        :param aggr: Message aggregation type, defaults to "mean"
        :type aggr: str, optional
        """
        super(DeltaNetAtomic, self).__init__()

        self.pos_dim = 3
        self.m_dim = m_dim
        self.embedding_dim = embedding_dim
        self.n_kernels = n_kernels
        self.n_mlp = n_mlp
        self.mlp_dim = mlp_dim
        self.n_outputs = n_outputs
        self.initialize_weights = initialize_weights
        self.fourier_features = fourier_features
        self.aggr = aggr

        # Embedding
        self.embedding = nn.Embedding(num_embeddings=11, embedding_dim=self.embedding_dim)

        # Kernel
        self.kernel_dim = self.embedding_dim
        self.kernels = nn.ModuleList()
        for _ in range(self.n_kernels):
            self.kernels.append(
                EGNN_sparse(
                    feats_dim=self.kernel_dim,
                    pos_dim=self.pos_dim,
                    m_dim=self.m_dim,
                    fourier_features=self.fourier_features,
                    aggr=self.aggr,
                )
            )

        # MLP
        self.fnn = nn.ModuleList()
        input_fnn = self.kernel_dim * (self.n_kernels + 1)
        self.fnn.append(nn.Linear(input_fnn, mlp_dim))
        for _ in range(self.n_mlp - 1):
            self.fnn.append(nn.Linear(self.mlp_dim, self.mlp_dim))
        self.fnn.append(nn.Linear(self.mlp_dim, self.n_outputs))

        # Initialize weights
        if self.initialize_weights:
            self.kernels.apply(weights_init)
            self.fnn.apply(weights_init)
            nn.init.xavier_uniform_(self.embedding.weight)

    def forward(self, g_batch):
        """_summary_

        :param g_batch: _description_
        :type g_batch: _type_
        :return: _description_
        :rtype: _type_
        """
        # Embedding
        features = self.embedding(g_batch.atomids)
        features = torch.cat([g_batch.coords, features], dim=1)

        # Kernel
        feature_list = []
        feature_list.append(features[:, self.pos_dim :])

        for kernel in self.kernels:
            features = kernel(
                x=features,
                edge_index=g_batch.edge_index,
            )
            feature_list.append(features[:, self.pos_dim :])

        # Concat
        features = F.silu(torch.cat(feature_list, dim=1))

        # MLP 1
        for mlp in self.fnn[:-1]:
            features = F.silu(mlp(features))

        # Outputlayer
        features = self.fnn[-1](features).squeeze(1)

        return features


def weights_init(m):
    """Xavier uniform weight initialization.

    :param m: A list of learnable linear PyTorch modules.
    :type m: [torch.nn.modules.linear.Linear]
    """
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)


def fourier_encode_dist(x, num_encodings=4, include_self=True):
    """Encoding Euclidian diatomic distances into Fourier features.

    :param x: Distances in Angström.
    :type x: Tensor
    :param num_encodings: Number of sine and cosine functions, defaults to 4
    :type num_encodings: int, optional
    :param include_self: Option to include absolute distance, defaults to True
    :type include_self: bool, optional
    :return: Fourier features.
    :rtype: Tensor
    """
    x = x.unsqueeze(-1)
    device, dtype, orig_x = x.device, x.dtype, x
    scales = 2 ** torch.arange(num_encodings, device=device, dtype=dtype)
    x = x / scales
    x = torch.cat([x.sin(), x.cos()], dim=-1)
    x = torch.cat((x, orig_x), dim=-1) if include_self else x
    return x


class EGNN_sparse(MessagePassing):
    """torch geometric message-passing layer for 3D molecular graphs."""

    def __init__(
        self, feats_dim, pos_dim=3, edge_attr_dim=0, m_dim=32, dropout=0.1, fourier_features=32, aggr="mean", **kwargs
    ):
        """Initialization of the 3D message passing layer.

        :param feats_dim: Node feature dimension.
        :type feats_dim: int
        :param pos_dim: Dimension of the graph, defaults to 3
        :type pos_dim: int, optional
        :param edge_attr_dim: Additional edge features (neglected in this implementation)
        :type edge_attr_dim: Tensor
        :param m_dim: Meessage passing feature dimesnion, defaults to 32
        :type m_dim: int, optional
        :param dropout: Dropout value, defaults to 0.1
        :type dropout: float, optional
        :param fourier_features: Number of Fourier features, defaults to 16
        :type fourier_features: int, optional
        :param aggr: Message aggregation type, defaults to "add"
        :type aggr: str, optional
        """
        assert aggr in {
            "add",
            "sum",
            "max",
            "mean",
        }, "pool method must be a valid option"

        kwargs.setdefault("aggr", aggr)
        super(EGNN_sparse, self).__init__(**kwargs)

        # Model parameters
        self.feats_dim = feats_dim
        self.pos_dim = pos_dim
        self.m_dim = m_dim
        self.fourier_features = fourier_features

        self.edge_input_dim = (self.fourier_features * 2) + edge_attr_dim + 1 + (feats_dim * 2)

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        # Edge layers
        self.edge_norm1 = nn.LayerNorm(m_dim)
        self.edge_norm2 = nn.LayerNorm(m_dim)

        self.edge_mlp = nn.Sequential(
            nn.Linear(self.edge_input_dim, self.edge_input_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.edge_input_dim * 2, m_dim),
            nn.SiLU(),
        )

        # Node layers
        self.node_norm1 = nn.LayerNorm(feats_dim)
        self.node_norm2 = nn.LayerNorm(feats_dim)

        self.node_mlp = nn.Sequential(
            nn.Linear(feats_dim + m_dim, feats_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(feats_dim * 2, feats_dim),
        )

        # Initialization
        self.apply(self.init_)

    def init_(self, module):
        if type(module) in {nn.Linear}:
            nn.init.xavier_normal_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
    ):
        """Forward pass in the mesaage passing fucntion.

        :param x: Node features.
        :type x: Tensor
        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        coors, feats = x[:, : self.pos_dim], x[:, self.pos_dim :]
        rel_coors = coors[edge_index[0]] - coors[edge_index[1]]
        rel_dist = (rel_coors**2).sum(dim=-1, keepdim=True)

        if self.fourier_features > 0:
            rel_dist = fourier_encode_dist(rel_dist, num_encodings=self.fourier_features)
            rel_dist = rearrange(rel_dist, "n () d -> n d")

        hidden_out = self.propagate(
            edge_index,
            x=feats,
            edge_attr=rel_dist,
            coors=coors,
            rel_coors=rel_coors,
        )

        return torch.cat([coors, hidden_out], dim=-1)

    def message(self, x_i, x_j, edge_attr):
        """Message passing.

        :param x_i: Node n_i.
        :type x_i: Tensor
        :param x_j: Node n_j.
        :type x_j: Tensor
        :param edge_attr: Edge e_{ij}
        :type edge_attr: Tensor
        :return: Message m_ji
        :rtype: Tensor
        """
        m_ij = self.edge_mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))
        return m_ij

    def propagate(self, edge_index: Adj, size: Size = None, **kwargs):
        """Overall propagation within the message passing.

        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        # get input tensors
        size = self._check_input(edge_index, size)
        coll_dict = self._collect(self._user_args, edge_index, size, kwargs)
        msg_kwargs = self.inspector.distribute("message", coll_dict)
        aggr_kwargs = self.inspector.distribute("aggregate", coll_dict)
        update_kwargs = self.inspector.distribute("update", coll_dict)

        # get messages
        m_ij = self.message(**msg_kwargs)
        m_ij = self.edge_norm1(m_ij)

        # aggregate messages
        m_i = self.aggregate(m_ij, **aggr_kwargs)
        m_i = self.edge_norm1(m_i)

        # get updated node features
        hidden_feats = self.node_norm1(kwargs["x"])
        hidden_out = self.node_mlp(torch.cat([hidden_feats, m_i], dim=-1))
        hidden_out = self.node_norm2(hidden_out)
        hidden_out = kwargs["x"] + hidden_out

        return self.update(hidden_out, **update_kwargs)

In [156]:
def get_model(gpu=True):
    config = configparser.ConfigParser()
    config.read('/content/drive/MyDrive/2024_modeling/ETHmodlab-lsfml-d0178f1/lsfml/qml/config_14000.ini')

    KERNEL_DIM = int(config["PARAMS"]["KERNEL_DIM"])
    KERNEL_NUM = int(config["PARAMS"]["KERNEL_NUM"])
    MLP_DIM = int(config["PARAMS"]["MLP_DIM"])
    MLP_NUM = int(config["PARAMS"]["MLP_NUM"])
    EDGE_DIM = int(config["PARAMS"]["EDGE_DIM"])
    OUTPUT_DIM = int(config["PARAMS"]["OUTPUT_DIM"])
    AGGR = str(config["PARAMS"]["AGGR"]).strip('"')
    FOURIER = int(config["PARAMS"]["FOURIER"])

    model = DeltaNetAtomic(
        embedding_dim=KERNEL_DIM,
        n_kernels=KERNEL_NUM,
        n_mlp=MLP_NUM,
        mlp_dim=MLP_DIM,
        n_outputs=OUTPUT_DIM,
        m_dim=EDGE_DIM,
        initialize_weights=True,
        fourier_features=FOURIER,
        aggr=AGGR,
    )

    model.load_state_dict(
        torch.load(
            '/content/drive/MyDrive/2024_modeling/ETHmodlab-lsfml-d0178f1/lsfml/qml/model1.pkl',
            map_location=torch.device("cpu"),
        )
    )
    if gpu:
        model = model.to(DEVICE)
        print(f"QML model has been sent to {DEVICE}")
    else:
        model = model.to("cpu")
        print("QML model has been sent to cpu")

    return model

In [157]:
get_model()

QML model has been sent to cuda


DeltaNetAtomic(
  (embedding): Embedding(11, 128)
  (kernels): ModuleList(
    (0-4): 5 x EGNN_sparse()
  )
  (fnn): ModuleList(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1-2): 2 x Linear(in_features=256, out_features=256, bias=True)
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)

##csv서 그래프객체 만들어보기 혹은 데이터 전처리 후 export

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [ ]:
rxn_ex_df = pd.read_csv('/content/drive/MyDrive/2024_modeling/ETHmodlab-lsfml-d0178f1/lsfml/data/experimental_rxndata.csv')
rxn_ex_df

,rxn_id,educt,rxn_scale_mol,rxn_temp_C,rxn_time_h,rxn_atm,rxn_c_moll,catalyst,catalyst_eq,ligand,ligand_eq,reagent,reagent_eq,solvent,solvent_ratio,yes_no,mono_bo,di_bo,non_bo,drug
0,ELN036496-175_1_1_D6,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=C(C(=C2C=CC3=C(N=CC(=C3C)C)C12)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
1,ELN036496-175_1_1_D5,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC=C2C=CC=3C=CC=NC3C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
2,ELN036496-175_1_1_D4,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
3,ELN036496-175_1_1_D3,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C(C)(C)C)C(C)(C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
4,ELN036496-175_1_1_D2,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N1=CC=CC2=CC=CC(N)=C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,ELN036496-112_1_1_A5,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC=C2C=CC=3C=CC=NC3C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.78,0.0,0.23,0
952,ELN036496-112_1_1_A4,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.82,0.0,0.18,0
953,ELN036496-112_1_1_A3,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C(C)(C)C)C(C)(C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.78,0.0,0.21,0
954,ELN036496-112_1_1_A2,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N1=CC=CC2=CC=CC(N)=C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.43,0.0,0.57,0


In [ ]:
from collections import Counter

def get_dict_for_embedding(list):
    """Creates a dictionary from a list of strings as keys and values form 0 to N, where N = len(list).

    :param list: List of strings.
    :type list:  list[str]
    :return: dictionary, mapping each string to an integer.
    :rtype: dirct[str] = int
    """

    list_dict = {}
    list_counter = Counter(list)

    for idx, x in enumerate(list_counter):
        list_dict[x] = idx

    return list_dict

In [ ]:
HYBRIDISATIONS = ["SP3","SP2","SP","UNSPECIFIED","S"]
AROMATOCITY = ["True","False"]
IS_RING = ["True","False"]
ATOMTYPES = ["H","C","N","O","F","P","S","Cl","Br","I"]
QML_ATOMTYPES = ["X","H","C","N","O","F","P","S","Cl","Br","I"]

In [ ]:
HYBRIDISATION_DICT = get_dict_for_embedding(HYBRIDISATIONS)
AROMATOCITY_DICT = get_dict_for_embedding(AROMATOCITY)
IS_RING_DICT = get_dict_for_embedding(IS_RING)
ATOMTYPE_DICT = get_dict_for_embedding(ATOMTYPES)
QML_ATOMTYPE_DICT = get_dict_for_embedding(QML_ATOMTYPES)

In [153]:
QMLMODEL = get_model(gpu=False)

RuntimeError: Error(s) in loading state_dict for DeltaNetAtomic:
	size mismatch for kernels.0.edge_mlp.0.weight: copying a param with shape torch.Size([642, 321]) from checkpoint, the shape in current model is torch.Size([512, 256]).
	size mismatch for kernels.0.edge_mlp.0.bias: copying a param with shape torch.Size([642]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for kernels.0.edge_mlp.3.weight: copying a param with shape torch.Size([32, 642]) from checkpoint, the shape in current model is torch.Size([32, 512]).
	size mismatch for kernels.1.edge_mlp.0.weight: copying a param with shape torch.Size([642, 321]) from checkpoint, the shape in current model is torch.Size([512, 256]).
	size mismatch for kernels.1.edge_mlp.0.bias: copying a param with shape torch.Size([642]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for kernels.1.edge_mlp.3.weight: copying a param with shape torch.Size([32, 642]) from checkpoint, th

In [ ]:
import h5py, os
import networkx as nx
import numpy as np
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops
from torch_geometric.utils.undirected import to_undirected
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform

def get_info_from_smi(smi, randomseed, radius):
    """Main function for extracting relevant reaction conditions and generating the 2D and 3D molecular graphs given a SMILES-string and a seed for 3D conformer generation.

    :param smi: SMILES-string
    :type smi: str
    :param randomseed: random seed
    :type randomseed: int
    :return: tuple including all graph-relevant numpy arrays
    :rtype: tuple
    """
    # Get mol objects from smiles
    mol_no_Hs = Chem.MolFromSmiles(smi)
    mol = Chem.rdmolops.AddHs(mol_no_Hs)

    atomids = []
    qml_atomids = []
    is_ring = []
    hyb = []
    arom = []
    crds_3d = []

    AllChem.EmbedMolecule(mol, randomSeed=randomseed)
    AllChem.UFFOptimizeMolecule(mol)

    for idx, i in enumerate(mol.GetAtoms()):
        atomids.append(ATOMTYPE_DICT[i.GetSymbol()])
        qml_atomids.append(QML_ATOMTYPE_DICT[i.GetSymbol()])
        is_ring.append(IS_RING_DICT[str(i.IsInRing())])
        hyb.append(HYBRIDISATION_DICT[str(i.GetHybridization())])
        arom.append(AROMATOCITY_DICT[str(i.GetIsAromatic())])
        crds_3d.append(list(mol.GetConformer().GetAtomPosition(idx)))

    atomids = np.array(atomids)
    qml_atomids = np.array(qml_atomids)
    is_ring = np.array(is_ring)
    hyb = np.array(hyb)
    arom = np.array(arom)
    crds_3d = np.array(crds_3d)

    # Edges for covalent bonds in sdf file
    edge_dir1 = []
    edge_dir2 = []
    for idx, bond in enumerate(mol.GetBonds()):
        a2 = bond.GetEndAtomIdx()
        a1 = bond.GetBeginAtomIdx()
        edge_dir1.append(a1)
        edge_dir1.append(a2)
        edge_dir2.append(a2)
        edge_dir2.append(a1)

    edge_2d = torch.from_numpy(np.array([edge_dir1, edge_dir2]))

    # 3D graph for qml and qml prediction
    qml_atomids = torch.LongTensor(qml_atomids)
    xyzs = torch.FloatTensor(crds_3d)
    edge_index = np.array(nx.complete_graph(qml_atomids.size(0)).edges())
    edge_index = to_undirected(torch.from_numpy(edge_index).t().contiguous())
    edge_index, _ = add_self_loops(edge_index, num_nodes=crds_3d.shape[0])

    qml_graph = Data(
        atomids=qml_atomids,
        coords=xyzs,
        edge_index=edge_index,
        num_nodes=qml_atomids.size(0),
    )

    charges = QMLMODEL(qml_graph).unsqueeze(1).detach().numpy()

    # Get edges for 3d graph
    distance_matrix = squareform(pdist(crds_3d))
    np.fill_diagonal(distance_matrix, float("inf"))  # to remove self-loops
    edge_3d = torch.from_numpy(np.vstack(np.where(distance_matrix <= radius)))

    return (
        atomids,
        is_ring,
        hyb,
        arom,
        charges,
        edge_2d,
        edge_3d,
        crds_3d,
    )

In [ ]:
get_info_from_smi(rxn_ex_df['educt'][0], 42, 4)

(array([1, 1, 1, 3, 2, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]),
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]),
 array([[-0.02108142],
        [-0.00772694],
        [ 0.0817143 ],
        [-0.18906604],
        [-0.02406343],
        [ 0.05932166],
        [-0.04225332],
        [-0.07643615],
        [ 0.30070752],
        [-0.22397198],
        [-0.07988009],
        [-0.00903304],
        [-0.00971619],
        [-0.05454497],
        [-0.10506798],
        [ 0.09800357],
        [-0.05137899],
        [-0.1470179 ],
        [ 0.05530123],
        [ 0.05701318],
        [ 0.04651501],
        [ 0.02806344],
        [ 0.02750515],
        [ 0.0233844 ],
        [ 0.02169081],
        [ 0.02988

In [ ]:
rxn_id = list(rxn_ex_df["rxn_id"])
educt = list(rxn_ex_df["educt"])
rxn_scale_mol = list(rxn_ex_df["rxn_scale_mol"])
rxn_temp_C = list(rxn_ex_df["rxn_temp_C"])
rxn_time_h = list(rxn_ex_df["rxn_time_h"])
rxn_atm = list(rxn_ex_df["rxn_atm"])
rxn_c_moll = list(rxn_ex_df["rxn_c_moll"])

    # Molecular conditions
catalyst = list(rxn_ex_df["catalyst"])
catalyst_eq = list(rxn_ex_df["catalyst_eq"])
ligand = list(rxn_ex_df["ligand"])
ligand_eq = list(rxn_ex_df["ligand_eq"])
reagent = list(rxn_ex_df["reagent"])
reagent_eq = list(rxn_ex_df["reagent_eq"])
solvent = list(rxn_ex_df["solvent"])
solvent_ratio = list(rxn_ex_df["solvent_ratio"])

# Targets
yes_no = list(rxn_ex_df["yes_no"])
mono_bo = list(rxn_ex_df["mono_bo"])
di_bo = list(rxn_ex_df["di_bo"])
non_bo = list(rxn_ex_df["non_bo"])

# Embedding of molecular conditions
rea_dict = get_dict_for_embedding(reagent)
lig_dict = get_dict_for_embedding(ligand)
cat_dict = get_dict_for_embedding(catalyst)
sol_dict = get_dict_for_embedding(solvent)

print("Liands in data set:", lig_dict)
print("Solvents in data set:", sol_dict)

Liands in data set: {'N=1C=C(C(=C2C=CC3=C(N=CC(=C3C)C)C12)C)C': 0, 'N=1C=CC=C2C=CC=3C=CC=NC3C12': 1, 'N=1C=CC(=CC1C=2N=CC=C(C2)C)C': 2, 'N=1C=CC(=CC1C=2N=CC=C(C2)C(C)(C)C)C(C)(C)C': 3, 'N1=CC=CC2=CC=CC(N)=C12': 4, 'N=1C=CC=CC1C=NN(CC=2C=CC=CC2)CC=3C=CC=CC3': 5}
Solvents in data set: {'N#CC': 0, 'O(C)C1CCCC1': 1, 'O1CCCC1C': 2, 'C1CCCCC1': 3}


In [ ]:
unique_substraes = {}

for idx, rxn_key in enumerate(rxn_id):
    short_rxn_key = rxn_key.split("_")[0]
    short_rxn_key = short_rxn_key.split("-")[-1]

    if short_rxn_key not in unique_substraes:
        unique_substraes[short_rxn_key] = educt[idx]

    else:
        pass

print(f"Calculating properties for {len(unique_substraes)} unique substartes")

Calculating properties for 40 unique substartes


In [ ]:
h5_path = '/content/drive/MyDrive/2024_modeling/240809_experimental_substrates.h5'

In [ ]:
with h5py.File(h5_path, "w") as lsf_container1:
    for rxn_key in tqdm(unique_substraes):
        (
            atom_id_a,
            ring_id_a,
            hybr_id_a,
            arom_id_a,
            charges_a,
            edge_2d_a,
            edge_3d_a,
            crds_3d_a,
        ) = get_info_from_smi(unique_substraes[rxn_key], 0xF00A, 4)

        (
                atom_id_b,
                ring_id_b,
                hybr_id_b,
                arom_id_b,
                charges_b,
                edge_2d_b,
                edge_3d_b,
                crds_3d_b,
        ) = get_info_from_smi(unique_substraes[rxn_key], 0xF00B, 4)

        (
                atom_id_c,
                ring_id_c,
                hybr_id_c,
                arom_id_c,
                charges_c,
                edge_2d_c,
                edge_3d_c,
                crds_3d_c,
        ) = get_info_from_smi(unique_substraes[rxn_key], 0xF00C, 4)

        (
                atom_id_d,
                ring_id_d,
                hybr_id_d,
                arom_id_d,
                charges_d,
                edge_2d_d,
                edge_3d_d,
                crds_3d_d,
        ) = get_info_from_smi(unique_substraes[rxn_key], 0xF00D, 4)

        (
                atom_id_e,
                ring_id_e,
                hybr_id_e,
                arom_id_e,
                charges_e,
                edge_2d_e,
                edge_3d_e,
                crds_3d_e,
        ) = get_info_from_smi(unique_substraes[rxn_key], 0xF00E, 4)

        lsf_container1.create_group(rxn_key)

        lsf_container1[rxn_key].create_dataset("atom_id_a", data=atom_id_a)
        lsf_container1[rxn_key].create_dataset("ring_id_a", data=ring_id_a)
        lsf_container1[rxn_key].create_dataset("hybr_id_a", data=hybr_id_a)
        lsf_container1[rxn_key].create_dataset("arom_id_a", data=arom_id_a)
        lsf_container1[rxn_key].create_dataset("charges_a", data=charges_a)
        lsf_container1[rxn_key].create_dataset("edge_2d_a", data=edge_2d_a)
        lsf_container1[rxn_key].create_dataset("edge_3d_a", data=edge_3d_a)
        lsf_container1[rxn_key].create_dataset("crds_3d_a", data=crds_3d_a)
        lsf_container1[rxn_key].create_dataset("atom_id_b", data=atom_id_b)
        lsf_container1[rxn_key].create_dataset("ring_id_b", data=ring_id_b)
        lsf_container1[rxn_key].create_dataset("hybr_id_b", data=hybr_id_b)
        lsf_container1[rxn_key].create_dataset("arom_id_b", data=arom_id_b)
        lsf_container1[rxn_key].create_dataset("charges_b", data=charges_b)
        lsf_container1[rxn_key].create_dataset("edge_2d_b", data=edge_2d_b)
        lsf_container1[rxn_key].create_dataset("edge_3d_b", data=edge_3d_b)
        lsf_container1[rxn_key].create_dataset("crds_3d_b", data=crds_3d_b)
        lsf_container1[rxn_key].create_dataset("atom_id_c", data=atom_id_c)
        lsf_container1[rxn_key].create_dataset("ring_id_c", data=ring_id_c)
        lsf_container1[rxn_key].create_dataset("hybr_id_c", data=hybr_id_c)
        lsf_container1[rxn_key].create_dataset("arom_id_c", data=arom_id_c)
        lsf_container1[rxn_key].create_dataset("charges_c", data=charges_c)
        lsf_container1[rxn_key].create_dataset("edge_2d_c", data=edge_2d_c)
        lsf_container1[rxn_key].create_dataset("edge_3d_c", data=edge_3d_c)
        lsf_container1[rxn_key].create_dataset("crds_3d_c", data=crds_3d_c)
        lsf_container1[rxn_key].create_dataset("atom_id_d", data=atom_id_d)
        lsf_container1[rxn_key].create_dataset("ring_id_d", data=ring_id_d)
        lsf_container1[rxn_key].create_dataset("hybr_id_d", data=hybr_id_d)
        lsf_container1[rxn_key].create_dataset("arom_id_d", data=arom_id_d)
        lsf_container1[rxn_key].create_dataset("charges_d", data=charges_d)
        lsf_container1[rxn_key].create_dataset("edge_2d_d", data=edge_2d_d)
        lsf_container1[rxn_key].create_dataset("edge_3d_d", data=edge_3d_d)
        lsf_container1[rxn_key].create_dataset("crds_3d_d", data=crds_3d_d)
        lsf_container1[rxn_key].create_dataset("atom_id_e", data=atom_id_e)
        lsf_container1[rxn_key].create_dataset("ring_id_e", data=ring_id_e)
        lsf_container1[rxn_key].create_dataset("hybr_id_e", data=hybr_id_e)
        lsf_container1[rxn_key].create_dataset("arom_id_e", data=arom_id_e)
        lsf_container1[rxn_key].create_dataset("charges_e", data=charges_e)
        lsf_container1[rxn_key].create_dataset("edge_2d_e", data=edge_2d_e)
        lsf_container1[rxn_key].create_dataset("edge_3d_e", data=edge_3d_e)
        lsf_container1[rxn_key].create_dataset("crds_3d_e", data=crds_3d_e)

    wins = 0
    loss = 0

    print(f"Transforming {len(rxn_id)} reactions into h5 format")

100%|██████████| 40/40 [00:21<00:00,  1.87it/s]


Transforming 956 reactions into h5 format


In [ ]:
atom_id_a == atom_id_b

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
from rdkit.Chem import rdMolDescriptors

def get_fp_from_smi(smi):
    """Calculates ECFP from SMILES-sting

    :param smi: SMILES string
    :type smi: str
    :return: ECFP fingerprint vector
    :rtype: np.ndarray
    """
    mol_no_Hs = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol_no_Hs)
    mfpgen = Chem.rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=256)

    return mfpgen.GetFingerprintAsNumPy(mol)

In [ ]:
with h5py.File("/content/drive/MyDrive/2024_modeling/240809_experimental_rxndata.h5", "w") as lsf_container:
    for idx, rxn_key in enumerate(tqdm(rxn_id)):
        try:
            rgnt_id = rea_dict[reagent[idx]]
            lgnd_id = lig_dict[ligand[idx]]
            clst_id = cat_dict[catalyst[idx]]
            slvn_id = sol_dict[solvent[idx]]

            # Create group in h5 for this id
            lsf_container.create_group(rxn_key)

            # Add all parameters as datasets to the created group
            if ligand_eq[idx] == "none":
                lgnd_eq = 0.0
            else:
                lgnd_eq = ligand_eq[idx]

            # Molecule
            ecfp4_2 = get_fp_from_smi(educt[idx])
            lsf_container[rxn_key].create_dataset("ecfp4_2", data=[ecfp4_2])

            # Conditions
            lsf_container[rxn_key].create_dataset("rgnt_id", data=[int(rgnt_id)])
            lsf_container[rxn_key].create_dataset("lgnd_id", data=[int(lgnd_id)])
            lsf_container[rxn_key].create_dataset("clst_id", data=[int(clst_id)])
            lsf_container[rxn_key].create_dataset("slvn_id", data=[int(slvn_id)])
            lsf_container[rxn_key].create_dataset("rgnt_eq", data=[float(reagent_eq[idx])])
            lsf_container[rxn_key].create_dataset("lgnd_eq", data=[float(lgnd_eq)])
            lsf_container[rxn_key].create_dataset("clst_eq", data=[float(catalyst_eq[idx])])
            lsf_container[rxn_key].create_dataset("rxn_scl", data=[float(rxn_scale_mol[idx])])
            lsf_container[rxn_key].create_dataset("rxn_con", data=[float(rxn_c_moll[idx])])
            lsf_container[rxn_key].create_dataset("rxn_tmp", data=[float(rxn_temp_C[idx])])
            lsf_container[rxn_key].create_dataset("rxn_tme", data=[float(rxn_time_h[idx])])

            # Tragets
            lsf_container[rxn_key].create_dataset("mono_id", data=[yes_no[idx]])
            lsf_container[rxn_key].create_dataset("mo_frct", data=[[mono_bo[idx]]])
            lsf_container[rxn_key].create_dataset("di_frct", data=[[di_bo[idx]]])
            lsf_container[rxn_key].create_dataset("no_frct", data=[[non_bo[idx]]])

            wins += 1

        except:
            loss += 1
    print(f"Reactions sucessfully transformed: {wins}; Reactions failed: {loss}")

100%|██████████| 956/956 [00:05<00:00, 178.74it/s]


Reactions sucessfully transformed: 1912; Reactions failed: 0


##Data and dataloader

In [22]:
from torch_geometric.data import Data
import random

class DataLSF(Dataset):
    def __init__(
        self,
        rxn_ids,
        data,
        data_substrates,
        target,
        graph_dim,
        fingerprint,
        conformers,
    ):
        self.target = target
        self.graph_dim = graph_dim
        self.fingerprint = fingerprint
        self.conformers = conformers
        self.rxn_ids = rxn_ids

        self.h5f = h5py.File(data)
        self.h5f_subs = h5py.File(data_substrates)

        nums = list(range(0, len(self.rxn_ids)))
        self.idx2rxn = {}
        for x in range(len(self.rxn_ids)):
            self.idx2rxn[nums[x]] = self.rxn_ids[x]

        print("\nLoader initialized:")
        print(f"Number of reactions loaded: {len(self.rxn_ids)}")
        print(f"Chosen target (binary or mono): {self.target}")
        print(f"Chosen graph_dim (edge_2d of edge_3d): {self.graph_dim}")
        print(f"Chosen fingerprint (ecfp4_2 of ecfp6_1): {self.fingerprint}")

    def __getitem__(self, idx):
        rxn_id = self.idx2rxn[idx]

        sbst_rxn = rxn_id.split("_")[0]
        sbst_rxn = sbst_rxn.split("-")[-1]

        # Pick random conformer
        conformer = random.choice(self.conformers)

        # Molecule
        atom_id = np.array(self.h5f_subs[sbst_rxn][f"atom_id_{conformer}"])
        ring_id = np.array(self.h5f_subs[sbst_rxn][f"ring_id_{conformer}"])
        hybr_id = np.array(self.h5f_subs[sbst_rxn][f"hybr_id_{conformer}"])
        arom_id = np.array(self.h5f_subs[sbst_rxn][f"arom_id_{conformer}"])
        charges = np.array(self.h5f_subs[sbst_rxn][f"charges_{conformer}"])
        crds_3d = np.array(self.h5f_subs[sbst_rxn][f"crds_3d_{conformer}"])

        if self.fingerprint is not None:
            ecfp_fp = np.array(self.h5f[rxn_id][self.fingerprint])
        else:
            ecfp_fp = np.array([])

        # Edge IDs with desired dimension
        edge_index = np.array(self.h5f_subs[sbst_rxn][f"{self.graph_dim}_{conformer}"])

        # Conditions
        lgnd_id = np.array(self.h5f[rxn_id]["lgnd_id"])
        slvn_id = np.array(self.h5f[rxn_id]["slvn_id"])

        # Tragets
        if self.target == "binary":
            rxn_trg = np.array(self.h5f[rxn_id]["mono_id"])
        elif self.target == "mono":
            mo_frct = np.array(self.h5f[rxn_id]["mo_frct"])
            di_frct = np.array(self.h5f[rxn_id]["di_frct"])
            rxn_trg = np.array([(mo_frct[i]+di_frct[i]) for i in range(len(mo_frct))])
            #rxn_trg = np.array(float(mo_frct) + float(di_frct)) #single element 꺼내기로 변경

        num_nodes = torch.LongTensor(atom_id).size(0)

        graph_data = Data(
            atom_id=torch.LongTensor(atom_id),
            ring_id=torch.LongTensor(ring_id),
            hybr_id=torch.LongTensor(hybr_id),
            arom_id=torch.LongTensor(arom_id),
            lgnd_id=torch.LongTensor(lgnd_id),
            slvn_id=torch.LongTensor(slvn_id),
            charges=torch.FloatTensor(charges),
            ecfp_fp=torch.FloatTensor(ecfp_fp),
            crds_3d=torch.FloatTensor(crds_3d),
            rxn_trg=torch.FloatTensor(rxn_trg),
            edge_index=torch.LongTensor(edge_index),
            num_nodes=num_nodes,
        )

        return graph_data

#그래프 객체를 따로 저장할 순 없을까?

    def __len__(self):
        """Return length

        :return: length
        :rtype: int
        """
        return len(self.rxn_ids)

In [ ]:
def create_fold(df, fold_seed, frac):

    train_frac, val_frac, test_frac = frac

    test = df.sample(frac=test_frac, random_state=fold_seed, replace=False)
    train_val = df.drop(test.index)

    val = train_val.sample(frac=val_frac/(train_frac+val_frac), random_state=fold_seed, replace=False)
    train = train_val.drop(val.index)

    return train, val, test

In [ ]:
train, val, test = create_fold(rxn_ex_df, 42, [0.7, 0.1, 0.2])

In [ ]:
train

,rxn_id,educt,rxn_scale_mol,rxn_temp_C,rxn_time_h,rxn_atm,rxn_c_moll,catalyst,catalyst_eq,ligand,ligand_eq,reagent,reagent_eq,solvent,solvent_ratio,yes_no,mono_bo,di_bo,non_bo,drug
0,ELN036496-175_1_1_D6,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=C(C(=C2C=CC3=C(N=CC(=C3C)C)C12)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
1,ELN036496-175_1_1_D5,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC=C2C=CC=3C=CC=NC3C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
2,ELN036496-175_1_1_D4,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
3,ELN036496-175_1_1_D3,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C(C)(C)C)C(C)(C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
4,ELN036496-175_1_1_D2,C1(=C(C)ON=C1C(C=C(F)C2)=CC=2)C3C=C[NH]N=3,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N1=CC=CC2=CC=CC(N)=C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,N#CC,1,0,0.00,0.0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,ELN036496-112_1_1_B5,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC=C2C=CC=3C=CC=NC3C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,O1CCCC1C,1,1,0.62,0.0,0.38,0
946,ELN036496-112_1_1_B4,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,O1CCCC1C,1,1,0.59,0.0,0.41,0
952,ELN036496-112_1_1_A4,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N=1C=CC(=CC1C=2N=CC=C(C2)C)C,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.82,0.0,0.18,0
954,ELN036496-112_1_1_A2,O(C1=CC=CC(OC)=C1OC)C,0.0001,80,18,N2,0.2,C[O+]1[Ir-]2[O+](C)[Ir-]12.C3CC=CCCC=C3.C4CC=C...,0.025,N1=CC=CC2=CC=CC(N)=C12,0.05,O1B(OC(C)(C)C1(C)C)B2OC(C)(C)C(O2)(C)C,1,C1CCCCC1,1,1,0.43,0.0,0.57,0


In [23]:
def get_rxn_ids(
    data,
    frac
):
    h5f = h5py.File(data)

     # Load all rxn keys
    rxn_ids = list(h5f.keys())
    random.shuffle(rxn_ids)
    train_frac, val_frac, test_frac = frac
    print(train_frac, val_frac, test_frac)

    train_ids = rxn_ids[: int(len(rxn_ids)*train_frac)]
    eval_ids = rxn_ids[int(len(rxn_ids)*train_frac) : (int(len(rxn_ids)*train_frac) + int(len(rxn_ids)*val_frac))]
    test_ids = rxn_ids[(int(len(rxn_ids)*train_frac) + int(len(rxn_ids)*val_frac)):]

    return train_ids, eval_ids, test_ids

In [24]:
train_ids, eval_ids, test_ids = get_rxn_ids(
            "/content/drive/MyDrive/2024_modeling/240809_experimental_rxndata.h5", [0.7, 0.1, 0.2])

0.7 0.1 0.2


In [ ]:
#GEOMETRY = True if GEOMETRY >= 1 else False
#GRAPH_DIM = "edge_3d" if GEOMETRY >= 1 else "edge_2d"
#FINGERPRINT = FINGERPRINT if args.mode == "c" else None

[PARAMS]  
LR_FACTOR = 1e-4  
LR_STEP_SIZE = 100  
N_KERNELS = 3  
POOLING_HEADS = 4  
D_MLP = 512  
D_KERNEL = 128  
D_EMBEDDING = 128  
BATCH_SIZE = 16  
SPLIT = random  
ELN = ELN036496-146  
TARGET = mono  
GEOMETRY = 1  
QML = 1  
FINGERPRINT = ecfp4_2

In [65]:
from torch_geometric.loader import DataLoader
CONFORMER_LIST = ["a", "b", "c", "d", "e"]

train_data = DataLSF(
            rxn_ids=train_ids,
            data="/content/drive/MyDrive/2024_modeling/240809_experimental_rxndata.h5",
            data_substrates="/content/drive/MyDrive/2024_modeling/240809_experimental_substrates.h5",
            target='mono',
            graph_dim='edge_2d',
            fingerprint=None,
            conformers=CONFORMER_LIST)


Loader initialized:
Number of reactions loaded: 669
Chosen target (binary or mono): mono
Chosen graph_dim (edge_2d of edge_3d): edge_2d
Chosen fingerprint (ecfp4_2 of ecfp6_1): None


In [66]:
train_loader = DataLoader(
            train_data,
            batch_size=16,
            shuffle=True,
            num_workers=2)

In [67]:
evl_data = DataLSF(
            rxn_ids=eval_ids,
            data="/content/drive/MyDrive/2024_modeling/240809_experimental_rxndata.h5",
            data_substrates="/content/drive/MyDrive/2024_modeling/240809_experimental_substrates.h5",
            target='mono',
            graph_dim='edge_2d',
            fingerprint=None,
            conformers=CONFORMER_LIST)

eval_loader = DataLoader(
            evl_data,
            batch_size=16,
            shuffle=True,
            num_workers=2)


Loader initialized:
Number of reactions loaded: 95
Chosen target (binary or mono): mono
Chosen graph_dim (edge_2d of edge_3d): edge_2d
Chosen fingerprint (ecfp4_2 of ecfp6_1): None


In [68]:
test_data = DataLSF(
            rxn_ids=test_ids,
            data="/content/drive/MyDrive/2024_modeling/240809_experimental_rxndata.h5",
            data_substrates="/content/drive/MyDrive/2024_modeling/240809_experimental_substrates.h5",
            target='mono',
            graph_dim='edge_2d',
            fingerprint=None,
            conformers=CONFORMER_LIST)

test_loader = DataLoader(
            test_data,
            batch_size=16,
            shuffle=True,
            num_workers=2)


Loader initialized:
Number of reactions loaded: 192
Chosen target (binary or mono): mono
Chosen graph_dim (edge_2d of edge_3d): edge_2d
Chosen fingerprint (ecfp4_2 of ecfp6_1): None


##model construction

In [29]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.typing import Adj, Size, Tensor
from typing import Optional

class EGNN_sparse3D(MessagePassing):
    """torch geometric message-passing layer for 3D molecular graphs."""

    def __init__(self, feats_dim, pos_dim=3, m_dim=32, dropout=0.1, fourier_features=16, aggr="add", **kwargs):
        """Initialization of the 3D message passing layer.

        :param feats_dim: Node feature dimension.
        :type feats_dim: int
        :param pos_dim: Dimension of the graph, defaults to 3
        :type pos_dim: int, optional
        :param m_dim: Meessage passing feature dimesnion, defaults to 32
        :type m_dim: int, optional
        :param dropout: Dropout value, defaults to 0.1
        :type dropout: float, optional
        :param fourier_features: Number of Fourier features, defaults to 16
        :type fourier_features: int, optional
        :param aggr: Message aggregation type, defaults to "add"
        :type aggr: str, optional
        """
        assert aggr in {
            "add",
            "sum",
            "max",
            "mean",
        }, "pool method must be a valid option"

        kwargs.setdefault("aggr", aggr)
        super(EGNN_sparse3D, self).__init__(**kwargs)

        # Model parameters
        self.feats_dim = feats_dim
        self.pos_dim = pos_dim
        self.m_dim = m_dim
        self.fourier_features = fourier_features

        self.edge_input_dim = (self.fourier_features * 2) + 1 + (feats_dim * 2)

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.edge_norm1 = nn.LayerNorm(m_dim)
        self.edge_norm2 = nn.LayerNorm(m_dim)

        self.edge_mlp = nn.Sequential(
            nn.Linear(self.edge_input_dim, self.edge_input_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.edge_input_dim * 2, m_dim),
            nn.SiLU(),
        )

        self.node_norm1 = nn.LayerNorm(feats_dim)
        self.node_norm2 = nn.LayerNorm(feats_dim)

        self.node_mlp = nn.Sequential(
            nn.Linear(feats_dim + m_dim, feats_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(feats_dim * 2, feats_dim),
        )

        self.apply(self.init_)

    def init_(self, module):
        if type(module) in {nn.Linear}:
            nn.init.xavier_normal_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
    ):
        """Forward pass in the mesaage passing fucntion.

        :param x: Node features.
        :type x: Tensor
        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        coors, feats = x[:, : self.pos_dim], x[:, self.pos_dim :]
        rel_coors = coors[edge_index[0]] - coors[edge_index[1]]
        rel_dist = (rel_coors**2).sum(dim=-1, keepdim=True)

        if self.fourier_features > 0:
            rel_dist = fourier_encode_dist(rel_dist, num_encodings=self.fourier_features)
            rel_dist = rel_dist.squeeze(1)
            # rel_dist = rearrange(rel_dist, "n () d -> n d")

        hidden_out = self.propagate(edge_index, x=feats, edge_attr=rel_dist)
        return torch.cat([coors, hidden_out], dim=-1)

    def message(self, x_i, x_j, edge_attr):
        """Message passing.

        :param x_i: Node n_i.
        :type x_i: Tensor
        :param x_j: Node n_j.
        :type x_j: Tensor
        :param edge_attr: Edge e_{ij}
        :type edge_attr: Tensor
        :return: Message m_ji
        :rtype: Tensor
        """
        m_ij = self.edge_mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))
        return m_ij

    def propagate(self, edge_index: Adj, size: Size = None, **kwargs):
        """Overall propagation within the message passing.

        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        # get input tensors
        size = self._check_input(edge_index, size)
        coll_dict = self._collect(self._user_args, edge_index, size, kwargs)
        msg_kwargs = self.inspector.distribute("message", coll_dict)
        aggr_kwargs = self.inspector.distribute("aggregate", coll_dict)
        update_kwargs = self.inspector.distribute("update", coll_dict)

        # get messages
        m_ij = self.message(**msg_kwargs)
        m_ij = self.edge_norm1(m_ij)

        # aggregate messages
        m_i = self.aggregate(m_ij, **aggr_kwargs)
        m_i = self.edge_norm2(m_i)

        # get updated node features
        hidden_feats = self.node_norm1(kwargs["x"])
        hidden_out = self.node_mlp(torch.cat([hidden_feats, m_i], dim=-1))
        hidden_out = self.node_norm2(hidden_out)
        hidden_out = kwargs["x"] + hidden_out

        return self.update((hidden_out), **update_kwargs)

In [30]:
import math
from typing import List, Optional, Tuple, Type

import torch
from torch import Tensor
from torch.nn import LayerNorm, Linear
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_batch


class MAB(torch.nn.Module):
    def __init__(
        self, dim_Q: int, dim_K: int, dim_V: int, num_heads: int, Conv: Optional[Type] = None, layer_norm: bool = False
    ):
        super().__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.layer_norm = layer_norm

        self.fc_q = Linear(dim_Q, dim_V)

        if Conv is None:
            self.layer_k = Linear(dim_K, dim_V)
            self.layer_v = Linear(dim_K, dim_V)
        else:
            self.layer_k = Conv(dim_K, dim_V)
            self.layer_v = Conv(dim_K, dim_V)

        if layer_norm:
            self.ln0 = LayerNorm(dim_V)
            self.ln1 = LayerNorm(dim_V)

        self.fc_o = Linear(dim_V, dim_V)

    def reset_parameters(self):
        self.fc_q.reset_parameters()
        self.layer_k.reset_parameters()
        self.layer_v.reset_parameters()
        if self.layer_norm:
            self.ln0.reset_parameters()
            self.ln1.reset_parameters()
        self.fc_o.reset_parameters()
        pass

    def forward(
        self,
        Q: Tensor,
        K: Tensor,
        graph: Optional[Tuple[Tensor, Tensor, Tensor]] = None,
        mask: Optional[Tensor] = None,
    ) -> Tensor:
        Q = self.fc_q(Q)

        if graph is not None:
            x, edge_index, batch = graph
            K, V = self.layer_k(x, edge_index), self.layer_v(x, edge_index)
            K, _ = to_dense_batch(K, batch)
            V, _ = to_dense_batch(V, batch)
        else:
            K, V = self.layer_k(K), self.layer_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), dim=0)
        K_ = torch.cat(K.split(dim_split, 2), dim=0)
        V_ = torch.cat(V.split(dim_split, 2), dim=0)

        if mask is not None:
            mask = torch.cat([mask for _ in range(self.num_heads)], 0)
            attention_score = Q_.bmm(K_.transpose(1, 2))
            attention_score = attention_score / math.sqrt(self.dim_V)
            A = torch.softmax(mask + attention_score, 1)
        else:
            A = torch.softmax(Q_.bmm(K_.transpose(1, 2)) / math.sqrt(self.dim_V), 1)

        out = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)

        if self.layer_norm:
            out = self.ln0(out)

        out = out + self.fc_o(out).relu()

        if self.layer_norm:
            out = self.ln1(out)

        return out


class SAB(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        num_heads: int,
        Conv: Optional[Type] = None,
        layer_norm: bool = False,
    ):
        super().__init__()
        self.mab = MAB(in_channels, in_channels, out_channels, num_heads, Conv=Conv, layer_norm=layer_norm)

    def reset_parameters(self):
        self.mab.reset_parameters()

    def forward(
        self,
        x: Tensor,
        graph: Optional[Tuple[Tensor, Tensor, Tensor]] = None,
        mask: Optional[Tensor] = None,
    ) -> Tensor:
        return self.mab(x, x, graph, mask)


class PMA(torch.nn.Module):
    def __init__(
        self, channels: int, num_heads: int, num_seeds: int, Conv: Optional[Type] = None, layer_norm: bool = False
    ):
        super().__init__()
        self.S = torch.nn.Parameter(torch.Tensor(1, num_seeds, channels))
        self.mab = MAB(channels, channels, channels, num_heads, Conv=Conv, layer_norm=layer_norm)

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.S)
        self.mab.reset_parameters()

    def forward(
        self,
        x: Tensor,
        graph: Optional[Tuple[Tensor, Tensor, Tensor]] = None,
        mask: Optional[Tensor] = None,
    ) -> Tensor:
        return self.mab(self.S.repeat(x.size(0), 1, 1), x, graph, mask)


class GraphMultisetTransformer(torch.nn.Module):
    r"""The global Graph Multiset Transformer pooling operator from the
    `"Accurate Learning of Graph Representations
    with Graph Multiset Pooling" <https://arxiv.org/abs/2102.11533>`_ paper.
    The Graph Multiset Transformer clusters nodes of the entire graph via
    attention-based pooling operations (:obj:`"GMPool_G"` or
    :obj:`"GMPool_I"`).
    In addition, self-attention (:obj:`"SelfAtt"`) can be used to calculate
    the inter-relationships among nodes.
    Args:
        in_channels (int): Size of each input sample.
        hidden_channels (int): Size of each hidden sample.
        out_channels (int): Size of each output sample.
        conv (Type, optional): A graph neural network layer
            for calculating hidden representations of nodes for
            :obj:`"GMPool_G"` (one of
            :class:`~torch_geometric.nn.conv.GCNConv`,
            :class:`~torch_geometric.nn.conv.GraphConv` or
            :class:`~torch_geometric.nn.conv.GATConv`).
            (default: :class:`~torch_geometric.nn.conv.GCNConv`)
        num_nodes (int, optional): The number of average
            or maximum nodes. (default: :obj:`300`)
        pooling_ratio (float, optional): Graph pooling ratio
            for each pooling. (default: :obj:`0.25`)
        pool_sequences ([str], optional): A sequence of pooling layers
            consisting of Graph Multiset Transformer submodules (one of
            :obj:`["GMPool_I"]`,
            :obj:`["GMPool_G"]`,
            :obj:`["GMPool_G", "GMPool_I"]`,
            :obj:`["GMPool_G", "SelfAtt", "GMPool_I"]` or
            :obj:`["GMPool_G", "SelfAtt", "SelfAtt", "GMPool_I"]`).
            (default: :obj:`["GMPool_G", "SelfAtt", "GMPool_I"]`)
        num_heads (int, optional): Number of attention heads.
            (default: :obj:`4`)
        layer_norm (bool, optional): If set to :obj:`True`, will make use of
            layer normalization. (default: :obj:`False`)
    """

    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        Conv: Optional[Type] = None,
        num_nodes: int = 300,
        pooling_ratio: float = 0.25,
        pool_sequences: List[str] = ["GMPool_G", "SelfAtt", "GMPool_I"],
        num_heads: int = 4,
        layer_norm: bool = False,
    ):
        super().__init__()
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.Conv = Conv or GCNConv
        self.num_nodes = num_nodes
        self.pooling_ratio = pooling_ratio
        self.pool_sequences = pool_sequences
        self.num_heads = num_heads
        self.layer_norm = layer_norm

        self.lin1 = Linear(in_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

        self.pools = torch.nn.ModuleList()
        num_out_nodes = math.ceil(num_nodes * pooling_ratio)
        for i, pool_type in enumerate(pool_sequences):
            if pool_type not in ["GMPool_G", "GMPool_I", "SelfAtt"]:
                raise ValueError(
                    "Elements in 'pool_sequences' should be one " "of 'GMPool_G', 'GMPool_I', or 'SelfAtt'"
                )

            if i == len(pool_sequences) - 1:
                num_out_nodes = 1

            if pool_type == "GMPool_G":
                self.pools.append(
                    PMA(hidden_channels, num_heads, num_out_nodes, Conv=self.Conv, layer_norm=layer_norm)
                )
                num_out_nodes = math.ceil(num_out_nodes * self.pooling_ratio)

            elif pool_type == "GMPool_I":
                self.pools.append(PMA(hidden_channels, num_heads, num_out_nodes, Conv=None, layer_norm=layer_norm))
                num_out_nodes = math.ceil(num_out_nodes * self.pooling_ratio)

            elif pool_type == "SelfAtt":
                self.pools.append(SAB(hidden_channels, hidden_channels, num_heads, Conv=None, layer_norm=layer_norm))

    def reset_parameters(self):
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()
        for pool in self.pools:
            pool.reset_parameters()

    def forward(self, x: Tensor, batch: Tensor, edge_index: Optional[Tensor] = None) -> Tensor:
        """"""
        x = self.lin1(x)
        batch_x, mask = to_dense_batch(x, batch)
        mask = (~mask).unsqueeze(1).to(dtype=x.dtype) * -1e9

        for i, (name, pool) in enumerate(zip(self.pool_sequences, self.pools)):
            graph = (x, edge_index, batch) if name == "GMPool_G" else None
            batch_x = pool(batch_x, graph, mask)
            mask = None

        return self.lin2(batch_x.squeeze(1))

    def __repr__(self) -> str:
        return (
            f"{self.__class__.__name__}({self.in_channels}, "
            f"{self.out_channels}, pool_sequences={self.pool_sequences})"
        )

In [31]:
def broadcast(src: torch.Tensor, other: torch.Tensor, dim: int):
    if dim < 0:
        dim = other.dim() + dim
    if src.dim() == 1:
        for _ in range(0, dim):
            src = src.unsqueeze(0)
    for _ in range(src.dim(), other.dim()):
        src = src.unsqueeze(-1)
    src = src.expand_as(other)
    return src

def scatter_sum(
    src: torch.Tensor,
    index: torch.Tensor,
    dim: int = -1,
    out: Optional[torch.Tensor] = None,
    dim_size: Optional[int] = None,
) -> torch.Tensor:
    index = broadcast(index, src, dim)
    if out is None:
        size = list(src.size())
        if dim_size is not None:
            size[dim] = dim_size
        elif index.numel() == 0:
            size[dim] = 0
        else:
            size[dim] = int(index.max()) + 1
        out = torch.zeros(size, dtype=src.dtype, device=src.device)
        return out.scatter_add_(dim, index, src)
    else:
        return out.scatter_add_(dim, index, src)

In [63]:
class EGNN_sparse(MessagePassing):
    """torch geometric message-passing layer for 2D molecular graphs."""

    def __init__(self, feats_dim, m_dim=32, dropout=0.1, aggr="add", **kwargs):
        """Initialization of the 2D message passing layer.

        :param feats_dim: Node feature dimension.
        :type feats_dim: int
        :param m_dim: Meessage passing feature dimesnion, defaults to 32
        :type m_dim: int, optional
        :param dropout: Dropout value, defaults to 0.1
        :type dropout: float, optional
        :param aggr: Message aggregation type, defaults to "add"
        :type aggr: str, optional
        """
        assert aggr in {
            "add",
            "sum",
            "max",
            "mean",
        }, "pool method must be a valid option"

        kwargs.setdefault("aggr", aggr)
        super(EGNN_sparse, self).__init__(**kwargs)

        self.feats_dim = feats_dim
        self.m_dim = m_dim

        self.edge_input_dim = feats_dim * 2

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.edge_norm1 = nn.LayerNorm(m_dim)
        self.edge_norm2 = nn.LayerNorm(m_dim)

        self.edge_mlp = nn.Sequential(
            nn.Linear(self.edge_input_dim, self.edge_input_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.edge_input_dim * 2, m_dim),
            nn.SiLU(),
        )

        self.node_norm1 = nn.LayerNorm(feats_dim)
        self.node_norm2 = nn.LayerNorm(feats_dim)

        self.node_mlp = nn.Sequential(
            nn.Linear(feats_dim + m_dim, feats_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(feats_dim * 2, feats_dim),
        )

        self.apply(self.init_)

    def init_(self, module):
        if type(module) in {nn.Linear}:
            nn.init.xavier_normal_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
    ):
        """Forward pass in the mesaage passing fucntion.

        :param x: Node features.
        :type x: Tensor
        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        hidden_out = self.propagate(edge_index, x=x)

        return hidden_out

    def message(self, x_i, x_j):
        """Message passing.

        :param x_i: Node n_i.
        :type x_i: Tensor
        :param x_j: Node n_j.
        :type x_j: Tensor
        :return: Message m_ji
        :rtype: Tensor
        """
        m_ij = self.edge_mlp(torch.cat([x_i, x_j], dim=-1))
        return m_ij

    def propagate(self, edge_index: Adj, size: Size = None, **kwargs):
        """Overall propagation within the message passing.

        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        # get input tensors
        size = self._check_input(edge_index, size)
        coll_dict = self._collect(self._user_args, edge_index, size, kwargs)
        msg_kwargs = self.inspector.distribute("message", coll_dict)
        aggr_kwargs = self.inspector.distribute("aggregate", coll_dict)
        update_kwargs = self.inspector.distribute("update", coll_dict)

        # get messages
        m_ij = self.message(**msg_kwargs)
        m_ij = self.edge_norm1(m_ij)

        # aggregate messages
        m_i = self.aggregate(m_ij, **aggr_kwargs)
        m_i = self.edge_norm2(m_i)

        # get updated node features
        hidden_feats = self.node_norm1(kwargs["x"])
        hidden_out = self.node_mlp(torch.cat([hidden_feats, m_i], dim=-1))
        hidden_out = self.node_norm2(hidden_out)
        hidden_out = kwargs["x"] + hidden_out

        return self.update((hidden_out), **update_kwargs)

In [32]:
class EGNN(nn.Module):
    """Graph neural network (GNN) using sum pooling for yield and binary predictions."""

    def __init__(self, n_kernels=3, mlp_dim=512, kernel_dim=64, embeddings_dim=64, qml=True, geometry=True):
        """Initialization of GNN

        :param n_kernels: Number of message passing functions, defaults to 3
        :type n_kernels: int, optional
        :param mlp_dim: Feature dimension within the multi layer perceptrons, defaults to 512
        :type mlp_dim: int, optional
        :param kernel_dim: Feature dimension within the message passing fucntions, defaults to 64
        :type kernel_dim: int, optional
        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions and atomic features), defaults to 64
        :type embeddings_dim: int, optional
        :param qml: Option to include DFT-level partial charges, defaults to True
        :type qml: bool, optional
        :param geometry: Option to include steric information in the input graph, defaults to True
        :type geometry: bool, optional
        """
        super(EGNN, self).__init__()

        self.embeddings_dim = embeddings_dim
        self.m_dim = 16
        self.kernel_dim = kernel_dim
        self.n_kernels = n_kernels
        self.aggr = "add"
        self.pos_dim = 3
        self.mlp_dim = mlp_dim
        self.qml = qml
        self.geometry = geometry

        dropout = 0.1
        self.dropout = nn.Dropout(dropout)

        self.atom_em = nn.Embedding(num_embeddings=10, embedding_dim=self.embeddings_dim)
        self.ring_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)
        self.hybr_em = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)
        self.arom_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)

        if self.qml:
            self.chrg_em = nn.Linear(1, self.embeddings_dim)
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 5
            self.chrg_em.apply(weights_init)
        else:
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 4

        self.pre_egnn_mlp = nn.Sequential(
            nn.Linear(self.pre_egnn_mlp_input_dim, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim),
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
        )

        self.kernels = nn.ModuleList()
        for _ in range(self.n_kernels):
            if self.geometry:
                self.kernels.append(
                    EGNN_sparse3D(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )
            else:
                self.kernels.append(
                    EGNN_sparse(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )

        self.post_egnn_mlp = nn.Sequential(
            nn.Linear(self.kernel_dim * self.n_kernels, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim * 2),
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim * 2),
            nn.SiLU(),
        )

        self.lig_emb = nn.Embedding(num_embeddings=6, embedding_dim=self.embeddings_dim)
        self.sol_emb = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)

        self.post_pooling_mlp_input_dim = self.kernel_dim * 2 + self.embeddings_dim * 2

        self.post_pooling_mlp = nn.Sequential(
            nn.Linear(self.post_pooling_mlp_input_dim, self.mlp_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, 1),
        )

        self.kernels.apply(weights_init)
        self.pre_egnn_mlp.apply(weights_init)
        self.post_egnn_mlp.apply(weights_init)
        self.post_pooling_mlp.apply(weights_init)
        nn.init.xavier_uniform_(self.atom_em.weight)
        nn.init.xavier_uniform_(self.ring_em.weight)
        nn.init.xavier_uniform_(self.hybr_em.weight)
        nn.init.xavier_uniform_(self.arom_em.weight)
        nn.init.xavier_uniform_(self.lig_emb.weight)
        nn.init.xavier_uniform_(self.sol_emb.weight)

    def forward(self, g_batch):
        """Forward pass of the GNN.

        :param g_batch: Input graph.
        :type g_batch: class
        :return: Prediction.
        :rtype: Tensor
        """
        if self.qml:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                        self.chrg_em(g_batch.charges),
                    ],
                    dim=1,
                )
            )
        else:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                    ],
                    dim=1,
                )
            )

        feature_list = []
        if self.geometry:
            features = torch.cat([g_batch.crds_3d, features], dim=1)
            for kernel in self.kernels:
                features = kernel(
                    x=features,
                    edge_index=g_batch.edge_index,
                )
                feature_list.append(features[:, self.pos_dim :])
        else:
            for kernel in self.kernels:
                features = kernel(x=features, edge_index=g_batch.edge_index)
                feature_list.append(features)

        features = torch.cat(feature_list, dim=1)
        features = self.post_egnn_mlp(features)

        del feature_list

        features = scatter_sum(features, g_batch.batch, dim=0)

        conditions = torch.cat(
            [
                self.lig_emb(g_batch.lgnd_id),
                self.sol_emb(g_batch.slvn_id),
            ],
            dim=1,
        )

        features = torch.cat([features, conditions], dim=1)
        features = self.post_pooling_mlp(features).squeeze(1)

        return features

In [ ]:
gnn3d_model = EGNN() #geometry만 true 즉 GNN3D

In [ ]:
gnn3d_model.to(DEVICE)

EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): SiLU()
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): SiLU()
  )
  (lig_emb): Embedding(6, 64)
  (sol_emb): Embedding(4, 64)
  (post_pooling_mlp): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
  

In [ ]:
gnn3dq_model = EGNN() #all_true 즉 GNN3DQM

In [ ]:
gnn2d_model = EGNN() #all_false 즉 GNN2D

In [ ]:
gnn2d_model.to(DEVICE)

EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): SiLU()
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): SiLU()
  )
  (lig_emb): Embedding(6, 64)
  (sol_emb): Embedding(4, 64)
  (post_pooling_mlp): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    

In [ ]:
gnn3dq_model.to(DEVICE)

EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): SiLU()
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): SiLU()
  )
  (lig_emb): Embedding(6, 64)
  (sol_emb): Embedding(4, 64)
  (post_pooling_mlp): Sequential(
 

In [ ]:
gnn2dq_model = EGNN() #qml만 true 즉 GNN2DQM

In [ ]:
gnn2dq_model.to(DEVICE)

EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): SiLU()
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): SiLU()
  )
  (lig_emb): Embedding(6, 64)
  (sol_emb): Embedding(4, 64)
  (post_pooling_mlp): Sequential(
   

In [33]:
class FNN(nn.Module):
    """Feed forward neural network (FNN) for yield and binary predictions."""

    def __init__(self, fp_dim=256, mlp_dim=512, kernel_dim=64, embeddings_dim=64):
        """Initialization of FNN

        :param fp_dim: Input dimension of the ECFP descriptor, defaults to 256
        :type fp_dim: int, optional
        :param mlp_dim: Feature dimension within the multi layer perceptrons, defaults to 512
        :type mlp_dim: int, optional
        :param kernel_dim: Feature dimension within the message passing fucntions, defaults to 64
        :type kernel_dim: int, optional
        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions), defaults to 64
        :type embeddings_dim: int, optional
        """
        super(FNN, self).__init__()

        self.embeddings_dim = embeddings_dim
        self.kernel_dim = kernel_dim
        self.mlp_dim = mlp_dim
        self.fp_dim = fp_dim

        dropout = 0.1
        self.dropout = nn.Dropout(dropout)

        self.pre_egnn_mlp = nn.Sequential(
            nn.Linear(self.fp_dim, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim * 2),
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim),
        )

        self.lig_emb = nn.Embedding(num_embeddings=6, embedding_dim=self.embeddings_dim)
        self.sol_emb = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)

        self.post_pooling_mlp_input_dim = self.kernel_dim + self.embeddings_dim * 2

        self.post_pooling_mlp = nn.Sequential(
            nn.Linear(self.post_pooling_mlp_input_dim, self.mlp_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, 1),
        )

        self.post_pooling_mlp.apply(weights_init)
        nn.init.xavier_uniform_(self.lig_emb.weight)
        nn.init.xavier_uniform_(self.sol_emb.weight)

    def forward(self, g_batch):
        """Forward pass of the FNN.

        :param g_batch: Input graph.
        :type g_batch: class
        :return: Prediction.
        :rtype: Tensor
        """
        features = self.pre_egnn_mlp(g_batch.ecfp_fp)

        conditions = torch.cat(
            [
                self.lig_emb(g_batch.lgnd_id),
                self.sol_emb(g_batch.slvn_id),
            ],
            dim=1,
        )

        features = torch.cat([features, conditions], dim=1)
        features = self.post_pooling_mlp(features).squeeze(1)

        return features

In [ ]:
fnn = FNN()

In [ ]:
fnn.to(DEVICE)

FNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): SiLU()
    (5): Linear(in_features=128, out_features=64, bias=True)
  )
  (lig_emb): Embedding(6, 64)
  (sol_emb): Embedding(4, 64)
  (post_pooling_mlp): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SiLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): SiLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [64]:
class GraphTransformer(nn.Module):
    """Graph Transformer neural network (GTNN) for yield and binary predictions."""

    def __init__(
        self,
        n_kernels=3,
        pooling_heads=8,
        mlp_dim=512,
        kernel_dim=64,
        embeddings_dim=64,
        qml=True,
        geometry=True,
    ):
        """Initialization of GTNN

        :param n_kernels: Number of message passing functions, defaults to 3
        :type n_kernels: int, optional
        :param pooling_heads: Number of Transformers, defaults to 8
        :type pooling_heads: int, optional
        :param mlp_dim: Feature dimension within the multi layer perceptrons, defaults to 512
        :type mlp_dim: int, optional
        :param kernel_dim: Feature dimension within the message passing fucntions, defaults to 64
        :type kernel_dim: int, optional
        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions and atomic features), defaults to 64
        :type embeddings_dim: int, optional
        :param qml: Option to include DFT-level partial charges, defaults to True
        :type qml: bool, optional
        :param geometry: Option to include steric information in the input graph, defaults to True
        :type geometry: bool, optional
        """
        super(GraphTransformer, self).__init__()

        self.embeddings_dim = embeddings_dim
        self.m_dim = 16
        self.kernel_dim = kernel_dim
        self.n_kernels = n_kernels
        self.aggr = "add"
        self.pos_dim = 3
        self.pooling_heads = pooling_heads
        self.mlp_dim = mlp_dim
        self.qml = qml
        self.geometry = geometry

        dropout = 0.1
        self.dropout = nn.Dropout(dropout)

        self.atom_em = nn.Embedding(num_embeddings=10, embedding_dim=self.embeddings_dim)
        self.ring_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)
        self.hybr_em = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)
        self.arom_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)

        if self.qml:
            self.chrg_em = nn.Linear(1, self.embeddings_dim)
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 5
            self.chrg_em.apply(weights_init)
        else:
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 4

        self.pre_egnn_mlp = nn.Sequential(
            nn.Linear(self.pre_egnn_mlp_input_dim, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim),
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
        )

        self.kernels = nn.ModuleList()
        for _ in range(self.n_kernels):
            if self.geometry:
                self.kernels.append(
                    EGNN_sparse3D(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )
            else:
                self.kernels.append(
                    EGNN_sparse(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )

        self.post_egnn_mlp = nn.Sequential(
            nn.Linear(self.kernel_dim * self.n_kernels, self.kernel_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
        )

        self.transformers = nn.ModuleList()
        for _ in range(self.pooling_heads):
            self.transformers.append(
                GraphMultisetTransformer(
                    in_channels=self.kernel_dim,
                    hidden_channels=self.kernel_dim,
                    out_channels=self.kernel_dim,
                    pool_sequences=["GMPool_G", "SelfAtt", "GMPool_I"],
                    num_heads=1,
                    layer_norm=True,
                )
            )

        self.lig_emb = nn.Embedding(num_embeddings=6, embedding_dim=self.embeddings_dim)
        self.sol_emb = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)

        self.post_pooling_mlp_input_dim = self.kernel_dim * (self.pooling_heads) + self.embeddings_dim * 2

        self.post_pooling_mlp = nn.Sequential(
            nn.Linear(self.post_pooling_mlp_input_dim, self.mlp_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, 1),
        )

        self.transformers.apply(weights_init)
        self.kernels.apply(weights_init)
        self.pre_egnn_mlp.apply(weights_init)
        self.post_egnn_mlp.apply(weights_init)
        self.post_pooling_mlp.apply(weights_init)
        nn.init.xavier_uniform_(self.atom_em.weight)
        nn.init.xavier_uniform_(self.ring_em.weight)
        nn.init.xavier_uniform_(self.hybr_em.weight)
        nn.init.xavier_uniform_(self.arom_em.weight)
        nn.init.xavier_uniform_(self.lig_emb.weight)
        nn.init.xavier_uniform_(self.sol_emb.weight)

    def forward(self, g_batch):
        """Forward pass of the GTNN.

        :param g_batch: Input graph.
        :type g_batch: class
        :return: Prediction.
        :rtype: Tensor
        """
        if self.qml:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                        self.chrg_em(g_batch.charges),
                    ],
                    dim=1,
                )
            )
        else:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                    ],
                    dim=1,
                )
            )

        feature_list = []
        if self.geometry:
            features = torch.cat([g_batch.crds_3d, features], dim=1)
            for kernel in self.kernels:
                features = kernel(
                    x=features,
                    edge_index=g_batch.edge_index,
                )
                feature_list.append(features[:, self.pos_dim :])
        else:
            for kernel in self.kernels:
                features = kernel(x=features, edge_index=g_batch.edge_index)
                feature_list.append(features)

        features = torch.cat(feature_list, dim=1)
        features = self.post_egnn_mlp(features)

        feature_list = []
        for transformer in self.transformers:
            feature_list.append(transformer(x=features, batch=g_batch.batch, edge_index=g_batch.edge_index))

        features = torch.cat(feature_list, dim=1)
        del feature_list

        conditions = torch.cat(
            [
                self.lig_emb(g_batch.lgnd_id),
                self.sol_emb(g_batch.slvn_id),
            ],
            dim=1,
        )

        features = torch.cat([features, conditions], dim=1)
        features = self.post_pooling_mlp(features).squeeze(1)

        return features

In [ ]:
gtnn3dq_model=GraphTransformer()

In [ ]:
gtnn3dq_model.to(DEVICE)

GraphTransformer(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (transformers): ModuleList(
    (0-7): 8 x GraphMultisetTransformer(64, 64, pool_seque

In [39]:
gtnn3d_model = GraphTransformer(qml=False)
gtnn3d_model.to(DEVICE)

GraphTransformer(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (transformers): ModuleList(
    (0-7): 8 x GraphMultisetTransformer(64, 64, pool_sequences=['GMPool_G', 'SelfAtt', 'GMPool_I'])
  )
  (lig_emb): Embe

In [72]:
gtnn2d_model = GraphTransformer(qml=False, geometry=False)
gtnn2d_model.to(DEVICE)

GraphTransformer(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (transformers): ModuleList(
    (0-7): 8 x GraphMultisetTransformer(64, 64, pool_sequences=['GMPool_G', 'SelfAtt', 'GMPool_I'])
  )
  (lig_emb): Embedd

In [81]:
gtnn2dq_model = GraphTransformer(qml=True, geometry=False)
gtnn2dq_model.to(DEVICE)

GraphTransformer(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (transformers): ModuleList(
    (0-7): 8 x GraphMultisetTransformer(64, 64, pool_sequenc

In [ ]:
class Atomistic_EGNN(nn.Module):
    """Atomistic graph neural network (aGNN) for regioselectivity predictions."""

    def __init__(self, n_kernels=3, mlp_dim=512, kernel_dim=64, embeddings_dim=64, qml=True, geometry=True):
        """Initialization of aGNN

        :param n_kernels: Number of message passing functions, defaults to 3
        :type n_kernels: int, optional
        :param mlp_dim: Feature dimension within the multi layer perceptrons, defaults to 512
        :type mlp_dim: int, optional
        :param kernel_dim: Feature dimension within the message passing fucntions, defaults to 64
        :type kernel_dim: int, optional
        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions and atomic features), defaults to 64
        :type embeddings_dim: int, optional
        :param qml: Option to include DFT-level partial charges, defaults to True
        :type qml: bool, optional
        :param geometry: Option to include steric information in the input graph, defaults to True
        :type geometry: bool, optional
        """
        super(Atomistic_EGNN, self).__init__()

        self.embeddings_dim = embeddings_dim
        self.m_dim = 16
        self.kernel_dim = kernel_dim
        self.n_kernels = n_kernels
        self.aggr = "add"
        self.pos_dim = 3
        self.mlp_dim = mlp_dim
        self.qml = qml
        self.geometry = geometry

        dropout = 0.1
        self.dropout = nn.Dropout(dropout)

        self.atom_em = nn.Embedding(num_embeddings=10, embedding_dim=self.embeddings_dim)
        self.ring_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)
        self.hybr_em = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)
        self.arom_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)

        if self.qml:
            self.chrg_em = nn.Linear(1, self.embeddings_dim)
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 5
            self.chrg_em.apply(weights_init)
        else:
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 4

        self.pre_egnn_mlp = nn.Sequential(
            nn.Linear(self.pre_egnn_mlp_input_dim, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim),
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
        )

        self.kernels = nn.ModuleList()
        for _ in range(self.n_kernels):
            if self.geometry:
                self.kernels.append(
                    EGNN_sparse3D(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )
            else:
                self.kernels.append(
                    EGNN_sparse(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )

        self.post_egnn_mlp = nn.Sequential(
            nn.Linear(self.kernel_dim * self.n_kernels, self.mlp_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, 1),
            nn.Sigmoid(),
        )

        self.kernels.apply(weights_init)
        self.pre_egnn_mlp.apply(weights_init)
        self.post_egnn_mlp.apply(weights_init)
        nn.init.xavier_uniform_(self.atom_em.weight)
        nn.init.xavier_uniform_(self.ring_em.weight)
        nn.init.xavier_uniform_(self.hybr_em.weight)
        nn.init.xavier_uniform_(self.arom_em.weight)

    def forward(self, g_batch):
        """Forward pass of the atomistic GNN.

        :param g_batch: Input graph.
        :type g_batch: class
        :return: Regioselectivity, 0 - 1 per atom.
        :rtype: Tensor
        """
        if self.qml:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                        self.chrg_em(g_batch.charges),
                    ],
                    dim=1,
                )
            )
        else:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                    ],
                    dim=1,
                )
            )

        feature_list = []
        if self.geometry:
            features = torch.cat([g_batch.crds_3d, features], dim=1)
            for kernel in self.kernels:
                features = kernel(
                    x=features,
                    edge_index=g_batch.edge_index,
                )
                feature_list.append(features[:, self.pos_dim :])
        else:
            for kernel in self.kernels:
                features = kernel(x=features, edge_index=g_batch.edge_index)
                feature_list.append(features)

        features = torch.cat(feature_list, dim=1)
        features = self.post_egnn_mlp(features).squeeze(1)

        return features

In [82]:
optimizer = torch.optim.Adam(
        gtnn2dq_model.parameters(),
        lr=1e-4,
        weight_decay=1e-10
    )
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=100,
        gamma=0.5)

In [37]:
def mae_loss(x, y):
    """Calculates the MAE loss.

    :param x: Predicted values.
    :type x: Tensor
    :param y: True values.
    :type y: Tensor
    :return: Calculated MAE
    :rtype: numpy.float64
    """
    return F.l1_loss(x, y).item()

##model train, val and test

In [83]:
epoches = 1000

for epoch in range(epoches):

    gtnn2dq_model.train()
    training_loss = []

    for g in train_loader:
        g = g.to(DEVICE)
        optimizer.zero_grad()
        #print(g.size)

        pred = gtnn2dq_model(g)

        loss = criterion(pred, g.rxn_trg.squeeze(1))
        loss.backward()
        optimizer.step()

    #scheduler.step()

        with torch.no_grad():
            mae = mae_loss(pred, g.rxn_trg.squeeze(1))
            training_loss.append(mae)
        #print(pred.shape, g.rxn_trg.squeeze(1).shape)
    #print(f'Epoch: {epoch+1:03d}, MAE: {np.mean(training_loss):.4f}')

    gtnn2dq_model.eval()
    eval_loss = []

    preds = []
    ys = []

    with torch.no_grad():
        for g in eval_loader:
            g = g.to(DEVICE)
            pred = gtnn2dq_model(g)
            mae = mae_loss(pred, g.rxn_trg.squeeze(1))
            eval_loss.append(mae)
            ys.append(g.rxn_trg)
            preds.append(pred)

    #scheduler.step()

    if (epoch+10) % 1 == 0:
        print(f'Epoch: {epoch:03d}, MAE(train): {np.mean(training_loss):.4f}, MAE(val): {np.mean(eval_loss):.4f}')

Epoch: 000, MAE(train): 0.2179, MAE(val): 0.2002
Epoch: 001, MAE(train): 0.2029, MAE(val): 0.1611
Epoch: 002, MAE(train): 0.1793, MAE(val): 0.1835
Epoch: 003, MAE(train): 0.1795, MAE(val): 0.1320
Epoch: 004, MAE(train): 0.1918, MAE(val): 0.2086
Epoch: 005, MAE(train): 0.1679, MAE(val): 0.1860
Epoch: 006, MAE(train): 0.1540, MAE(val): 0.1215
Epoch: 007, MAE(train): 0.1568, MAE(val): 0.1802
Epoch: 008, MAE(train): 0.1434, MAE(val): 0.0962
Epoch: 009, MAE(train): 0.1297, MAE(val): 0.0910
Epoch: 010, MAE(train): 0.1260, MAE(val): 0.0920
Epoch: 011, MAE(train): 0.1195, MAE(val): 0.0893
Epoch: 012, MAE(train): 0.1246, MAE(val): 0.1045
Epoch: 013, MAE(train): 0.1184, MAE(val): 0.0747
Epoch: 014, MAE(train): 0.1086, MAE(val): 0.1465
Epoch: 015, MAE(train): 0.1059, MAE(val): 0.0881
Epoch: 016, MAE(train): 0.1042, MAE(val): 0.0787
Epoch: 017, MAE(train): 0.1001, MAE(val): 0.0993
Epoch: 018, MAE(train): 0.0964, MAE(val): 0.0868
Epoch: 019, MAE(train): 0.0989, MAE(val): 0.0682
Epoch: 020, MAE(trai

In [84]:
#test model
gtnn2dq_model.eval()
test_loss = []

test_preds = []
test_ys = []

with torch.no_grad():
    for g in test_loader:
        g = g.to(DEVICE)
        pred = gtnn2dq_model(g)
        mae = mae_loss(pred, g.rxn_trg.squeeze(1))
        test_loss.append(mae)
        test_ys.append(g.rxn_trg)
        test_preds.append(pred)

print(np.mean(test_loss))

0.052181695393907525


##save model

In [85]:
#model save
torch.save(obj=gtnn2dq_model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f='/content/drive/MyDrive/2024_modeling/240811_GTNN2DQM.pth')

##Regioselectivity

In [86]:
HYBRIDISATION_DICT = {"SP3": 0, "SP2": 1, "SP": 2, "UNSPECIFIED": 3, "S": 3}
AROMATOCITY_DICT = get_dict_for_embedding(AROMATOCITY)
IS_RING_DICT = get_dict_for_embedding(IS_RING)
QML_ATOMTYPE_DICT = get_dict_for_embedding(QML_ATOMTYPES)
ATOMTYPE_DICT = {"H": 0, "C": 1, "N": 2, "O": 3, "F": 4, "P": 5, "S": 6, "Cl": 7, "Br": 8, "I": 9}

In [137]:
repl = Chem.MolFromSmiles("[2H]")
patt = Chem.MolFromSmarts("B2OC(C)(C)C(O2)(C)C")

[04:37:56] WARNING: not removing hydrogen atom without neighbors


In [159]:
import h5py, os
import networkx as nx
import numpy as np
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops
from torch_geometric.utils.undirected import to_undirected
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform

def get_rms(smi, patt, repl):
    """Takes a SMILES-sting and replaces a substructure (patt) such as a functional group with another substructure (repl)

    :param smi: SMILES-string.
    :type smi: str
    :param patt: Pattern to be replaced.
    :type patt: str
    :param repl: Pattern to replace.
    :type repl: str
    :return: SMILES-string.
    :rtype: str
    """
    m = Chem.MolFromSmiles(smi)
    if m is None:
        return None
    else:
        rms = Chem.MolToSmiles(AllChem.ReplaceSubstructs(m, patt, repl)[0])

    m = Chem.MolFromSmiles(rms)
    while rms != Chem.MolToSmiles(AllChem.ReplaceSubstructs(m, patt, repl)[0]):
        print(
            f"More than one Boron: {rms == Chem.MolToSmiles(AllChem.ReplaceSubstructs(m, patt, repl)[0])}"
            f", {rms}, {Chem.MolToSmiles(AllChem.ReplaceSubstructs(m, patt, repl)[0])}"
        )
        rms = Chem.MolToSmiles(AllChem.ReplaceSubstructs(m, patt, repl)[0])
        m = Chem.MolFromSmiles(rms)

    return rms

def get_regioselectivity(rms, seed, radius):
    """Main function to generate the 2D and 3D molecular graphs and to extract the reagioselectivity given a SMILES-string and a seed for 3D conformer generation.

    :param rms: SMILES-string
    :type rms: str
    :param seed: Random seed for 3D conformer generation
    :type seed: int
    :return: tuple including all graph-relevant numpy arrays
    :rtype: tuple
    """
    mol_no_Hs = Chem.MolFromSmiles(rms)
    mol = Chem.rdmolops.AddHs(mol_no_Hs)

    atomids = []
    qml_atomids = []
    is_ring = []
    hyb = []
    arom = []
    crds_3d = []
    pot_trg = []

    AllChem.EmbedMolecule(mol, randomSeed=seed)
    AllChem.UFFOptimizeMolecule(mol)

    for idx, i in enumerate(mol.GetAtoms()):
        atomids.append(ATOMTYPE_DICT[i.GetSymbol()])
        qml_atomids.append(QML_ATOMTYPE_DICT[i.GetSymbol()])
        is_ring.append(IS_RING_DICT[str(i.IsInRing())])
        hyb.append(HYBRIDISATION_DICT[str(i.GetHybridization())])
        arom.append(AROMATOCITY_DICT[str(i.GetIsAromatic())])
        crds_3d.append(list(mol.GetConformer().GetAtomPosition(idx)))

        nghbrs = [x.GetSymbol() for x in i.GetNeighbors()]
        if (i.GetSymbol() == "C") and ("H" in nghbrs):
            pot_trg.append(1)
        else:
            pot_trg.append(0)

    trg_atoms = []
    edge_dir1 = []
    edge_dir2 = []
    for idx, bond in enumerate(mol.GetBonds()):
        a2 = bond.GetEndAtomIdx()
        a1 = bond.GetBeginAtomIdx()
        edge_dir1.append(a1)
        edge_dir1.append(a2)
        edge_dir2.append(a2)
        edge_dir2.append(a1)

        if mol.GetAtoms()[a1].GetIsotope() == 2:
            trg_atoms.append(a2)

        if mol.GetAtoms()[a2].GetIsotope() == 2:
            trg_atoms.append(a1)

    edge_2d = torch.from_numpy(np.array([edge_dir1, edge_dir2]))

    target = [0 for i, x in enumerate(atomids)]
    for trg_atom in trg_atoms:
        target[trg_atom] = 1

    atomids = np.array(atomids)
    target = np.array(target)
    qml_atomids = np.array(qml_atomids)
    is_ring = np.array(is_ring)
    hyb = np.array(hyb)
    arom = np.array(arom)
    crds_3d = np.array(crds_3d)
    pot_trg = np.array(pot_trg)

    # 3D graph for qml and qml prediction
    qml_atomids = torch.LongTensor(qml_atomids)
    xyzs = torch.FloatTensor(crds_3d)
    edge_index = np.array(nx.complete_graph(qml_atomids.size(0)).edges())
    edge_index = to_undirected(torch.from_numpy(edge_index).t().contiguous())
    edge_index, _ = add_self_loops(edge_index, num_nodes=crds_3d.shape[0])

    qml_graph = Data(
        atomids=qml_atomids,
        coords=xyzs,
        edge_index=edge_index,
        num_nodes=qml_atomids.size(0),
    )

    charges = QMLMODEL(qml_graph).unsqueeze(1).detach().numpy()

    # Get edges for 3d graph
    distance_matrix = squareform(pdist(crds_3d))
    np.fill_diagonal(distance_matrix, float("inf"))  # to remove self-loops
    edge_3d = torch.from_numpy(np.vstack(np.where(distance_matrix <= radius)))

    return (
        atomids,
        is_ring,
        hyb,
        arom,
        charges,
        edge_2d,
        edge_3d,
        crds_3d,
        pot_trg,
        target,
    )

In [ ]:
get_regioselectivity

In [93]:
df_regio = pd.read_csv('/content/drive/MyDrive/2024_modeling/ETHmodlab-lsfml-d0178f1/lsfml/data/literature_rxndata.csv', encoding="unicode_escape")
df_regio.head()

,rxn_id,source_id,rxn_type,temperature_degC,time_h,atmosphere,scale_mol,concentration_moll,stmat_1_smiles,stmat_1_eq,...,solvent_1_smiles,solvent_1_fraction,solvent_2_smiles,solvent_2_fraction,product_1_smiles,product_1_yield,product_1_yieldtype,product_2_smiles,product_2_yield,product_2_yieldtype
0,lit_pub_bo_1,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.15,GC,NaN,NaN,NaN
1,lit_pub_bo_2,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.42,GC,NaN,NaN,NaN
2,lit_pub_bo_3,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.23,GC,NaN,NaN,NaN
3,lit_pub_bo_4,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.55,GC,NaN,NaN,NaN
4,lit_pub_bo_5,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.33,GC,NaN,NaN,NaN


In [160]:
smiles = list(df_regio["product_1_smiles"])
rxn_yield = list(df_regio["product_1_yield"])

print(f"Initial number of reactions: {len(smiles)}")
rxn_yield = [0.0 if np.isnan(x) else x for x in rxn_yield]
smiles = [s for i, s in enumerate(smiles) if rxn_yield[i] >= 0.3]
print(f"Number of reactions after removing yields <= 30%: {len(smiles)}")
smiles = [s for i, s in enumerate(smiles) if len(str(smiles[i])) >= 5]
print(f"Number of reactions after removing nan SMILES: {len(smiles)}")
uniques = list(set(smiles))
print(f"Number of reactions after removing duplicate SMILES: {len(uniques)}")

Initial number of reactions: 1301
Number of reactions after removing yields <= 30%: 1071
Number of reactions after removing nan SMILES: 1069
Number of reactions after removing duplicate SMILES: 685


In [161]:
for smi in tqdm(uniques):
    try:
        rms = get_rms(smi, patt, repl)
        print(rms)
        if "[2H]" in rms:
            (atom_id,ring_id,hybr_id,arom_id,charges,edge_2d,edge_3d,crds_3d,pot_trg,reg_trg,) = get_regioselectivity(rms, 42, 4)
            print(atom_id)
        else:
            print(f"No boron in product or unconventional boron in product: {rms}, {smi}")
    except:
        continue

  0%|          | 1/685 [00:00<02:12,  5.16it/s]

[2H]c1cccc(-c2ccccc2)c1NC(=O)C(C)(C)C
[0 1 1 1 1 1 1 1 1 1 1 1 1 2 1 3 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[2H]c1cc(C)c2ccccc2c1-c1nccc2ccccc12


  1%|          | 4/685 [00:00<00:48, 14.07it/s]

[0 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2H]c1cc2ccccc2cc1O
[0 1 1 1 1 1 1 1 1 1 1 3 0 0 0 0 0 0 0]
[2H]c1cn([Si](C)(C)C(C)(C)C)c2ncccc12
[2H]c1ccc(C(F)(F)F)c(C(=O)N(CCCCCC)CCCCCC)c1


  1%|▏         | 10/685 [00:00<00:40, 16.77it/s]

...

[0 1 1 1 1 7 1 1 1 2 2 1 1 0 0 0 0 0 0 0 0 0 0]
[2H]C1CCC1(CCc1ccc(F)cc1)C(=O)Nc1c(F)c(F)c(C(F)(F)F)c(F)c1F
[0 1 1 1 1 1 1 1 1 1 1 4 1 1 1 3 2 1 1 4 1 4 1 1 4 4 4 1 4 1 4 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
[2H]CC(C)C(=O)Nc1c(F)c(F)c(C(F)(F)F)c(F)c1F
[0 1 1 1 1 3 2 1 1 4 1 4 1 1 4 4 4 1 4 1 4 0 0 0 0 0 0 0]
[2H]c1cccc(N(C)C(=O)C(F)(F)F)c1


 99%|█████████▉| 681/685 [00:37<00:00, 23.75it/s]

[0 1 1 1 1 1 2 1 1 3 1 4 4 4 1 0 0 0 0 0 0 0]
[2H]c1cc(Br)ccc1NC(=O)C(C)(C)C
[0 1 1 1 8 1 1 1 2 1 3 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2H]c1cc(Cl)cc2cc(C)[nH]c12
[0 1 1 1 7 1 1 1 1 1 2 1 0 0 0 0 0 0 0]
[2H]c1cc(OC)ccc1SC
[0 1 1 1 3 1 1 1 1 6 1 0 0 0 0 0 0 0 0 0]
[2H]c1cc(C)cc2cc[nH]c12
[0 1 1 1 1 1 1 1 1 2 1 0 0 0 0 0 0 0 0]
[2H]c1ccc(C(=O)N(C(C)C)C(C)C)o1
[0 1 1 1 1 1 3 2 1 1 1 1 1 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2H]c1ccnc(C(=O)OCC)c1


100%|██████████| 685/685 [00:38<00:00, 17.99it/s]

[0 1 1 1 2 1 1 3 3 1 1 1 0 0 0 0 0 0 0 0]
[2H]c1cc(I)cc2cc[nH]c12
[0 1 1 1 9 1 1 1 1 2 1 0 0 0 0 0]
[2H]c1cccc2c1[nH]c1ccccc12
[0 1 1 1 1 1 1 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
[2H]c1ccc([Si](C)(C)C)cc1


In [162]:
wins = 0
loss = 0
rxn_key = 0

all_smiles = []
h5_path = '/content/drive/MyDrive/2024_modeling/240811_literature_regio.h5'

In [158]:
QMLMODEL = get_model(gpu=False)

QML model has been sent to cpu


In [163]:
with h5py.File(h5_path, "w") as lsf_container:
    for smi in tqdm(uniques):
        try:
            rms = get_rms(smi, patt, repl)
            if rms is not None:
                #print(rms)
                if "[2H]" in rms:
                    (atom_id,ring_id,hybr_id,arom_id,charges,edge_2d,edge_3d,crds_3d,pot_trg,reg_trg,) = get_regioselectivity(rms, 42, 4)
                    #print(atom_id)

                    # Create group in h5 for this id
                    lsf_container.create_group(str(rxn_key))

                    # Molecule
                    lsf_container[str(rxn_key)].create_dataset("atom_id", data=atom_id)
                    lsf_container[str(rxn_key)].create_dataset("ring_id", data=ring_id)
                    lsf_container[str(rxn_key)].create_dataset("hybr_id", data=hybr_id)
                    lsf_container[str(rxn_key)].create_dataset("arom_id", data=arom_id)
                    lsf_container[str(rxn_key)].create_dataset("edge_2d", data=edge_2d)
                    lsf_container[str(rxn_key)].create_dataset("edge_3d", data=edge_3d)
                    lsf_container[str(rxn_key)].create_dataset("charges", data=charges)
                    lsf_container[str(rxn_key)].create_dataset("crds_3d", data=crds_3d)
                    lsf_container[str(rxn_key)].create_dataset("pot_trg", data=pot_trg)
                    lsf_container[str(rxn_key)].create_dataset("reg_trg", data=reg_trg)

                    all_smiles.append(rms)

                    wins += 1
                    rxn_key += 1

                else:
                    loss += 1
                    print(f"No boron in product or unconventional boron in product: {rms}, {smi}")
            else:
                loss += 1
                print(f"No valid SMILES: {smi}")

        except:
            loss += 1

print(f"Reactions sucessfully transformed: {wins}; Reactions failed {loss}")

  4%|▍         | 28/685 [00:01<00:38, 17.05it/s]

No valid SMILES: [O-]1[B+3]2([O-]C(C)(C)C1(C)C)[C-]=3C=CC=CC3C=4C=CC=C[N]42


 14%|█▍        | 96/685 [00:05<00:26, 22.60it/s]

No valid SMILES: [O-]1[B+3]2([O-]C(C)(C)C1(C)C)[C-]=3C=CC=4C=CC=CC4C3C5=CC(=CC=[N]52)C


 16%|█▋        | 112/685 [00:06<00:26, 21.62it/s]

...

No valid SMILES: [O-]1[B+3]2([O-]C(C)(C)C1(C)C)[C-]=3C=CC=4C=CC=CC4C3C5=CC=C(C=[N]52)C


 92%|█████████▏| 631/685 [00:38<00:02, 21.93it/s]

More than one Boron: False, [2H]c1ccc(C#N)c2cc(B3OC(C)(C)C(C)(C)O3)[nH]c12, [2H]c1cc2c(C#N)ccc([2H])c2[nH]1
No boron in product or unconventional boron in product: CC(C)c1ccc2c(c1)CCC1C(CO)(C(=O)Nc3c(F)c(F)c(C(F)(F)F)c(F)c3F)CCCC21C, O=C(NC1=C(F)C(F)=C(C(F)=C1F)C(F)(F)F)C2(CO)CCCC3(C4=CC=C(C=C4CCC32)C(C)C)C


100%|██████████| 685/685 [00:40<00:00, 16.76it/s]


Reactions sucessfully transformed: 652; Reactions failed 33


In [164]:
rxn_key

652

In [165]:
all_smiles

['[2H]c1cccc(-c2ccccc2)c1NC(=O)C(C)(C)C',
 '[2H]c1cc(C)c2ccccc2c1-c1nccc2ccccc12',
 '[2H]c1cc2ccccc2cc1O',
 '[2H]c1ccc(C(F)(F)F)c(C(=O)N(CCCCCC)CCCCCC)c1',
 '[2H]c1cc(Br)cc(C(=O)N(C(C)C)C(C)C)c1Cl',
 '[2H]c1ccc(C)cc1F',
 '[2H]c1ccc(OC)c(CCCNC(=O)C(F)(F)F)c1',
 '[2H]c1cc(OC)cc2nc(C)oc12',
 '[2H]c1cc(Cl)cc(SC)c1',
 '[2H]c1ccc(CCCN(CC)C(=O)C(F)(F)F)c(Br)c1',
 '[2H]c1cc(C=Cc2ccccc2)cc2c1c(C(=O)C(C)(C)C)cn2S(=O)(=O)c1ccc(C)cc1',
 '[2H]c1ccc(C(=O)OCC)c(OC)c1',
 '[2H]c1cccc(C(F)(F)F)c1C(=O)Nc1c(F)c(F)c(C(F)(F)F)c(F)c1F',
 '[2H]c1cc(I)cc2ccn(C(=O)C(C)(C)C)c12',
 '[2H]c1cc(Cl)cc(OC)c1',
 '[2H]c1cc(OC)ccc1C(=O)NC',
 '[2H]c1c[nH]c2ccc(C)cc12',
 '[2H]c1cc(C=O)ccc1C#N',
 '[2H]c1ccc(Br)c2cc[nH]c12',
 '[2H]C1CCC1(CN1C(=O)c2ccccc2C1=O)C(=O)Nc1c(F)c(F)c(C(F)(F)F)c(F)c1F',
 '[2H]CCC(=O)Nc1c(F)c(F)c(C(F)(F)F)c(F)c1F',
 '[2H]c1cc(C)nc2cc(C#N)ccc12',
 '[2H]c1ccc(C(=O)OCC)c(F)c1F',
 '[2H]c1ccc(OC)c(P(=O)(C2CCCCC2)C2CCCCC2)c1',
 '[2H]c1ccc(CCCNC(=O)C(F)(F)F)c(OC)c1',
 '[2H]c1ccc(Cl)cc1C(=O)Nc1c(F)c(F)c(C(F)(

In [166]:
train_ids, eval_ids, test_ids = get_rxn_ids(
            "/content/drive/MyDrive/2024_modeling/240811_literature_regio.h5", [0.7, 0.1, 0.2])

0.7 0.1 0.2


In [167]:
train_ids

['412',
 '285',
 '532',
 '94',
 '284',
 '582',
 '133',
 '557',
 '606',
 '6',
 '493',
 '150',
 '162',
 '314',
 '295',
 '296',
 '92',
 '379',
 '499',
 '343',
 '155',
 '113',
 '234',
 '132',
 '232',
 '637',
 '229',
 '200',
 '63',
 '609',
 '268',
 '206',
 '347',
 '308',
 '438',
 '17',
 '477',
 '299',
 '14',
 '318',
 '72',
 '306',
 '65',
 '23',
 '370',
 '332',
 '471',
 '7',
 '248',
 '334',
 '418',
 '167',
 '5',
 '236',
 '492',
 '356',
 '374',
 '522',
 '160',
 '479',
 '4',
 '516',
 '124',
 '528',
 '362',
 '263',
 '592',
 '91',
 '585',
 '445',
 '26',
 '565',
 '324',
 '402',
 '217',
 '574',
 '386',
 '89',
 '566',
 '607',
 '129',
 '305',
 '413',
 '515',
 '36',
 '639',
 '355',
 '547',
 '145',
 '134',
 '177',
 '190',
 '194',
 '612',
 '437',
 '646',
 '322',
 '633',
 '191',
 '564',
 '505',
 '342',
 '537',
 '300',
 '218',
 '192',
 '539',
 '79',
 '373',
 '527',
 '265',
 '496',
 '495',
 '521',
 '149',
 '463',
 '56',
 '78',
 '50',
 '185',
 '277',
 '298',
 '3',
 '204',
 '442',
 '617',
 '116',
 '257',
 '

In [173]:
class DataLSF(Dataset):
    """Generates the desired graph objects (2D, 3D, QM) from reading the h5 files."""

    def __init__(
        self,
        rxn_ids,
        data,
        graph_dim,
    ):
        """Initialization.

        :param rxn_ids: Reaction IDs from the given split (train, eval, test)
        :type rxn_ids: list[str]
        :param data: Path to h5 file, including preprocessed data, defaults to "../../data/literature_regio.h5"
        :type data: str, optional
        :param graph_dim: Indicating 2D or 3D graph structure ("edge_2d" or "edge_3d"), defaults to "edge_2d"
        :type graph_dim: str, optional
        """
        # Define inputs
        self.graph_dim = graph_dim
        self.rxn_ids = rxn_ids

        # Load data from h5 file
        self.h5f = h5py.File(data)

        # Generate dict (int to rxn keys)
        nums = list(range(0, len(self.rxn_ids)))
        self.idx2rxn = {}
        for x in range(len(self.rxn_ids)):
            self.idx2rxn[nums[x]] = self.rxn_ids[x]

        print("\nLoader initialized:")
        print(f"Number of reactions loaded: {len(self.rxn_ids)}")
        print(f"Chosen graph_dim (edge_2d of edge_3d): {self.graph_dim}")

    def __getitem__(self, idx):
        """Loop over data.

        :param idx: Reaction ID
        :type idx: str
        :return: Input graph for the neural network.
        :rtype: torch_geometric.loader.dataloader.DataLoader
        """
        # int to rxn_id
        rxn_id = self.idx2rxn[idx]

        # Molecule
        atom_id = np.array(self.h5f[str(rxn_id)]["atom_id"])
        ring_id = np.array(self.h5f[str(rxn_id)]["ring_id"])
        hybr_id = np.array(self.h5f[str(rxn_id)]["hybr_id"])
        arom_id = np.array(self.h5f[str(rxn_id)]["arom_id"])
        charges = np.array(self.h5f[str(rxn_id)]["charges"])
        crds_3d = np.array(self.h5f[str(rxn_id)]["crds_3d"])
        pot_trg = np.array(self.h5f[str(rxn_id)]["pot_trg"])
        # print(idx, rxn_id, atom_id)

        # Edge IDs with desired dimension
        edge_index = np.array(self.h5f[str(rxn_id)][self.graph_dim])

        # Tragets
        rxn_trg = np.array(self.h5f[str(rxn_id)]["reg_trg"])

        num_nodes = torch.LongTensor(atom_id).size(0)

        graph_data = Data(
            atom_id=torch.LongTensor(atom_id),
            pot_trg=torch.LongTensor(pot_trg),
            ring_id=torch.LongTensor(ring_id),
            hybr_id=torch.LongTensor(hybr_id),
            arom_id=torch.LongTensor(arom_id),
            charges=torch.FloatTensor(charges),
            crds_3d=torch.FloatTensor(crds_3d),
            rxn_trg=torch.FloatTensor(rxn_trg),
            edge_index=torch.LongTensor(edge_index),
            num_nodes=num_nodes,
            rxn_id=rxn_id,
        )

        return graph_data

    def __len__(self):
        """Get length

        :return: length
        :rtype: int
        """
        return len(self.rxn_ids)

In [197]:
train_data = DataLSF(rxn_ids=train_ids,data='/content/drive/MyDrive/2024_modeling/240811_literature_regio.h5',graph_dim='edge_2d')
train_loader = DataLoader(train_data,batch_size=16,shuffle=True,num_workers=2)
eval_data = DataLSF(rxn_ids=eval_ids,data='/content/drive/MyDrive/2024_modeling/240811_literature_regio.h5',graph_dim='edge_2d')
eval_loader = DataLoader(eval_data,batch_size=16,shuffle=True,num_workers=2)
test_data = DataLSF(rxn_ids=test_ids,data='/content/drive/MyDrive/2024_modeling/240811_literature_regio.h5',graph_dim='edge_2d')
test_loader = DataLoader(test_data,batch_size=16,shuffle=True,num_workers=2)


Loader initialized:
Number of reactions loaded: 456
Chosen graph_dim (edge_2d of edge_3d): edge_2d

Loader initialized:
Number of reactions loaded: 65
Chosen graph_dim (edge_2d of edge_3d): edge_2d

Loader initialized:
Number of reactions loaded: 131
Chosen graph_dim (edge_2d of edge_3d): edge_2d


In [175]:
class EGNN_sparse(MessagePassing):
    """torch geometric message-passing layer for 2D molecular graphs."""

    def __init__(self, feats_dim, m_dim=32, dropout=0.1, aggr="add", **kwargs):
        """Initialization of the 2D message passing layer.

        :param feats_dim: Node feature dimension.
        :type feats_dim: int
        :param m_dim: Meessage passing feature dimesnion, defaults to 32
        :type m_dim: int, optional
        :param dropout: Dropout value, defaults to 0.1
        :type dropout: float, optional
        :param aggr: Message aggregation type, defaults to "add"
        :type aggr: str, optional
        """
        assert aggr in {
            "add",
            "sum",
            "max",
            "mean",
        }, "pool method must be a valid option"

        kwargs.setdefault("aggr", aggr)
        super(EGNN_sparse, self).__init__(**kwargs)

        self.feats_dim = feats_dim
        self.m_dim = m_dim

        self.edge_input_dim = feats_dim * 2

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.edge_norm1 = nn.LayerNorm(m_dim)
        self.edge_norm2 = nn.LayerNorm(m_dim)

        self.edge_mlp = nn.Sequential(
            nn.Linear(self.edge_input_dim, self.edge_input_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.edge_input_dim * 2, m_dim),
            nn.SiLU(),
        )

        self.node_norm1 = nn.LayerNorm(feats_dim)
        self.node_norm2 = nn.LayerNorm(feats_dim)

        self.node_mlp = nn.Sequential(
            nn.Linear(feats_dim + m_dim, feats_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(feats_dim * 2, feats_dim),
        )

        self.apply(self.init_)

    def init_(self, module):
        if type(module) in {nn.Linear}:
            nn.init.xavier_normal_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
    ):
        """Forward pass in the mesaage passing fucntion.

        :param x: Node features.
        :type x: Tensor
        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        hidden_out = self.propagate(edge_index, x=x)

        return hidden_out

    def message(self, x_i, x_j):
        """Message passing.

        :param x_i: Node n_i.
        :type x_i: Tensor
        :param x_j: Node n_j.
        :type x_j: Tensor
        :return: Message m_ji
        :rtype: Tensor
        """
        m_ij = self.edge_mlp(torch.cat([x_i, x_j], dim=-1))
        return m_ij

    def propagate(self, edge_index: Adj, size: Size = None, **kwargs):
        """Overall propagation within the message passing.

        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        # get input tensors
        size = self._check_input(edge_index, size)
        coll_dict = self._collect(self._user_args, edge_index, size, kwargs)
        msg_kwargs = self.inspector.distribute("message", coll_dict)
        aggr_kwargs = self.inspector.distribute("aggregate", coll_dict)
        update_kwargs = self.inspector.distribute("update", coll_dict)

        # get messages
        m_ij = self.message(**msg_kwargs)
        m_ij = self.edge_norm1(m_ij)

        # aggregate messages
        m_i = self.aggregate(m_ij, **aggr_kwargs)
        m_i = self.edge_norm2(m_i)

        # get updated node features
        hidden_feats = self.node_norm1(kwargs["x"])
        hidden_out = self.node_mlp(torch.cat([hidden_feats, m_i], dim=-1))
        hidden_out = self.node_norm2(hidden_out)
        hidden_out = kwargs["x"] + hidden_out

        return self.update((hidden_out), **update_kwargs)


def fourier_encode_dist(x, num_encodings=4, include_self=True):
    """Encoding Euclidian diatomic distances into Fourier features.

    :param x: Distances in Angström.
    :type x: Tensor
    :param num_encodings: Number of sine and cosine functions, defaults to 4
    :type num_encodings: int, optional
    :param include_self: Option to include absolute distance, defaults to True
    :type include_self: bool, optional
    :return: Fourier features.
    :rtype: Tensor
    """
    x = x.unsqueeze(-1)
    device, dtype, orig_x = x.device, x.dtype, x
    scales = 2 ** torch.arange(num_encodings, device=device, dtype=dtype)
    x = x / scales
    x = torch.cat([x.sin(), x.cos()], dim=-1)
    x = torch.cat((x, orig_x), dim=-1) if include_self else x
    return x


class EGNN_sparse3D(MessagePassing):
    """torch geometric message-passing layer for 3D molecular graphs."""

    def __init__(self, feats_dim, pos_dim=3, m_dim=32, dropout=0.1, fourier_features=16, aggr="add", **kwargs):
        """Initialization of the 3D message passing layer.

        :param feats_dim: Node feature dimension.
        :type feats_dim: int
        :param pos_dim: Dimension of the graph, defaults to 3
        :type pos_dim: int, optional
        :param m_dim: Meessage passing feature dimesnion, defaults to 32
        :type m_dim: int, optional
        :param dropout: Dropout value, defaults to 0.1
        :type dropout: float, optional
        :param fourier_features: Number of Fourier features, defaults to 16
        :type fourier_features: int, optional
        :param aggr: Message aggregation type, defaults to "add"
        :type aggr: str, optional
        """
        assert aggr in {
            "add",
            "sum",
            "max",
            "mean",
        }, "pool method must be a valid option"

        kwargs.setdefault("aggr", aggr)
        super(EGNN_sparse3D, self).__init__(**kwargs)

        # Model parameters
        self.feats_dim = feats_dim
        self.pos_dim = pos_dim
        self.m_dim = m_dim
        self.fourier_features = fourier_features

        self.edge_input_dim = (self.fourier_features * 2) + 1 + (feats_dim * 2)

        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.edge_norm1 = nn.LayerNorm(m_dim)
        self.edge_norm2 = nn.LayerNorm(m_dim)

        self.edge_mlp = nn.Sequential(
            nn.Linear(self.edge_input_dim, self.edge_input_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.edge_input_dim * 2, m_dim),
            nn.SiLU(),
        )

        self.node_norm1 = nn.LayerNorm(feats_dim)
        self.node_norm2 = nn.LayerNorm(feats_dim)

        self.node_mlp = nn.Sequential(
            nn.Linear(feats_dim + m_dim, feats_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(feats_dim * 2, feats_dim),
        )

        self.apply(self.init_)

    def init_(self, module):
        if type(module) in {nn.Linear}:
            nn.init.xavier_normal_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
    ):
        """Forward pass in the mesaage passing fucntion.

        :param x: Node features.
        :type x: Tensor
        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        coors, feats = x[:, : self.pos_dim], x[:, self.pos_dim :]
        rel_coors = coors[edge_index[0]] - coors[edge_index[1]]
        rel_dist = (rel_coors**2).sum(dim=-1, keepdim=True)

        if self.fourier_features > 0:
            rel_dist = fourier_encode_dist(rel_dist, num_encodings=self.fourier_features)
            rel_dist = rel_dist.squeeze(1)
            # rel_dist = rearrange(rel_dist, "n () d -> n d")

        hidden_out = self.propagate(edge_index, x=feats, edge_attr=rel_dist)
        return torch.cat([coors, hidden_out], dim=-1)

    def message(self, x_i, x_j, edge_attr):
        """Message passing.

        :param x_i: Node n_i.
        :type x_i: Tensor
        :param x_j: Node n_j.
        :type x_j: Tensor
        :param edge_attr: Edge e_{ij}
        :type edge_attr: Tensor
        :return: Message m_ji
        :rtype: Tensor
        """
        m_ij = self.edge_mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))
        return m_ij

    def propagate(self, edge_index: Adj, size: Size = None, **kwargs):
        """Overall propagation within the message passing.

        :param edge_index: Edge indices.
        :type edge_index: Adj
        :return: Updated node features.
        :rtype: Tensor
        """
        # get input tensors
        size = self._check_input(edge_index, size)
        coll_dict = self._collect(self._user_args, edge_index, size, kwargs)
        msg_kwargs = self.inspector.distribute("message", coll_dict)
        aggr_kwargs = self.inspector.distribute("aggregate", coll_dict)
        update_kwargs = self.inspector.distribute("update", coll_dict)

        # get messages
        m_ij = self.message(**msg_kwargs)
        m_ij = self.edge_norm1(m_ij)

        # aggregate messages
        m_i = self.aggregate(m_ij, **aggr_kwargs)
        m_i = self.edge_norm2(m_i)

        # get updated node features
        hidden_feats = self.node_norm1(kwargs["x"])
        hidden_out = self.node_mlp(torch.cat([hidden_feats, m_i], dim=-1))
        hidden_out = self.node_norm2(hidden_out)
        hidden_out = kwargs["x"] + hidden_out

        return self.update((hidden_out), **update_kwargs)

In [176]:
class Atomistic_EGNN(nn.Module):
    """Atomistic graph neural network (aGNN) for regioselectivity predictions."""

    def __init__(self, n_kernels=3, mlp_dim=512, kernel_dim=64, embeddings_dim=64, qml=True, geometry=True):
        """Initialization of aGNN

        :param n_kernels: Number of message passing functions, defaults to 3
        :type n_kernels: int, optional
        :param mlp_dim: Feature dimension within the multi layer perceptrons, defaults to 512
        :type mlp_dim: int, optional
        :param kernel_dim: Feature dimension within the message passing fucntions, defaults to 64
        :type kernel_dim: int, optional
        :param embeddings_dim: Embedding dimension of the input features (e.g. reaction conditions and atomic features), defaults to 64
        :type embeddings_dim: int, optional
        :param qml: Option to include DFT-level partial charges, defaults to True
        :type qml: bool, optional
        :param geometry: Option to include steric information in the input graph, defaults to True
        :type geometry: bool, optional
        """
        super(Atomistic_EGNN, self).__init__()

        self.embeddings_dim = embeddings_dim
        self.m_dim = 16
        self.kernel_dim = kernel_dim
        self.n_kernels = n_kernels
        self.aggr = "add"
        self.pos_dim = 3
        self.mlp_dim = mlp_dim
        self.qml = qml
        self.geometry = geometry

        dropout = 0.1
        self.dropout = nn.Dropout(dropout)

        self.atom_em = nn.Embedding(num_embeddings=10, embedding_dim=self.embeddings_dim)
        self.ring_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)
        self.hybr_em = nn.Embedding(num_embeddings=4, embedding_dim=self.embeddings_dim)
        self.arom_em = nn.Embedding(num_embeddings=2, embedding_dim=self.embeddings_dim)

        if self.qml:
            self.chrg_em = nn.Linear(1, self.embeddings_dim)
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 5
            self.chrg_em.apply(weights_init)
        else:
            self.pre_egnn_mlp_input_dim = self.embeddings_dim * 4

        self.pre_egnn_mlp = nn.Sequential(
            nn.Linear(self.pre_egnn_mlp_input_dim, self.kernel_dim * 2),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.kernel_dim * 2, self.kernel_dim),
            nn.SiLU(),
            nn.Linear(self.kernel_dim, self.kernel_dim),
            nn.SiLU(),
        )

        self.kernels = nn.ModuleList()
        for _ in range(self.n_kernels):
            if self.geometry:
                self.kernels.append(
                    EGNN_sparse3D(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )
            else:
                self.kernels.append(
                    EGNN_sparse(
                        feats_dim=self.kernel_dim,
                        m_dim=self.m_dim,
                        aggr=self.aggr,
                    )
                )

        self.post_egnn_mlp = nn.Sequential(
            nn.Linear(self.kernel_dim * self.n_kernels, self.mlp_dim),
            self.dropout,
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, self.mlp_dim),
            nn.SiLU(),
            nn.Linear(self.mlp_dim, 1),
            nn.Sigmoid(),
        )

        self.kernels.apply(weights_init)
        self.pre_egnn_mlp.apply(weights_init)
        self.post_egnn_mlp.apply(weights_init)
        nn.init.xavier_uniform_(self.atom_em.weight)
        nn.init.xavier_uniform_(self.ring_em.weight)
        nn.init.xavier_uniform_(self.hybr_em.weight)
        nn.init.xavier_uniform_(self.arom_em.weight)

    def forward(self, g_batch):
        """Forward pass of the atomistic GNN.

        :param g_batch: Input graph.
        :type g_batch: class
        :return: Regioselectivity, 0 - 1 per atom.
        :rtype: Tensor
        """
        if self.qml:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                        self.chrg_em(g_batch.charges),
                    ],
                    dim=1,
                )
            )
        else:
            features = self.pre_egnn_mlp(
                torch.cat(
                    [
                        self.atom_em(g_batch.atom_id),
                        self.ring_em(g_batch.ring_id),
                        self.hybr_em(g_batch.hybr_id),
                        self.arom_em(g_batch.arom_id),
                    ],
                    dim=1,
                )
            )

        feature_list = []
        if self.geometry:
            features = torch.cat([g_batch.crds_3d, features], dim=1)
            for kernel in self.kernels:
                features = kernel(
                    x=features,
                    edge_index=g_batch.edge_index,
                )
                feature_list.append(features[:, self.pos_dim :])
        else:
            for kernel in self.kernels:
                features = kernel(x=features, edge_index=g_batch.edge_index)
                feature_list.append(features)

        features = torch.cat(feature_list, dim=1)
        features = self.post_egnn_mlp(features).squeeze(1)

        return features

In [186]:
agnn3dq_model=Atomistic_EGNN()
agnn3dq_model.to(DEVICE)

Atomistic_EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SiLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): SiLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

In [193]:
agnn3d_model=Atomistic_EGNN(qml=False)
agnn3d_model.to(DEVICE)

Atomistic_EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse3D()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SiLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): SiLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

In [201]:
agnn2d_model=Atomistic_EGNN(qml=False, geometry=False)
agnn2d_model.to(DEVICE)

Atomistic_EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SiLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): SiLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

In [208]:
agnn2dq_model=Atomistic_EGNN(qml=True, geometry=False)
agnn2dq_model.to(DEVICE)

Atomistic_EGNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (atom_em): Embedding(10, 64)
  (ring_em): Embedding(2, 64)
  (hybr_em): Embedding(4, 64)
  (arom_em): Embedding(2, 64)
  (chrg_em): Linear(in_features=1, out_features=64, bias=True)
  (pre_egnn_mlp): Sequential(
    (0): Linear(in_features=320, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=64, bias=True)
    (6): SiLU()
  )
  (kernels): ModuleList(
    (0-2): 3 x EGNN_sparse()
  )
  (post_egnn_mlp): Sequential(
    (0): Linear(in_features=192, out_features=512, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SiLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): SiLU()
    (7): Linear(in_features=512, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

In [209]:
optimizer = torch.optim.Adam(
        agnn2dq_model.parameters(),
        lr=1e-4,
        weight_decay=1e-10,
    )
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=100,
        gamma=0.5
    )

In [210]:
epoches = 1000

for epoch in range(epoches):

    agnn2dq_model.train()
    training_loss = []

    for g in train_loader:
        g = g.to(DEVICE)
        optimizer.zero_grad()
        #print(g.size)

        pred = agnn2dq_model(g)

        loss = criterion(pred, g.rxn_trg)
        loss.backward()
        optimizer.step()

    #scheduler.step()

        with torch.no_grad():
            mae = mae_loss(pred, g.rxn_trg)
            training_loss.append(mae)
        #print(pred.shape, g.rxn_trg.squeeze(1).shape)
    #print(f'Epoch: {epoch+1:03d}, MAE: {np.mean(training_loss):.4f}')

    agnn2dq_model.eval()
    eval_loss = []

    preds = []
    ys = []

    with torch.no_grad():
        for g in eval_loader:
            g = g.to(DEVICE)
            pred = agnn2dq_model(g)
            mae = mae_loss(pred, g.rxn_trg)
            eval_loss.append(mae)
            ys.append(g.rxn_trg)
            preds.append(pred)

    #scheduler.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch: {epoch:03d}, MAE(train): {np.mean(training_loss):.4f}, MAE(val): {np.mean(eval_loss):.4f}')

Epoch: 009, MAE(train): 0.0634, MAE(val): 0.0633
Epoch: 019, MAE(train): 0.0614, MAE(val): 0.0632
Epoch: 029, MAE(train): 0.0594, MAE(val): 0.0586
Epoch: 039, MAE(train): 0.0575, MAE(val): 0.0573
Epoch: 049, MAE(train): 0.0545, MAE(val): 0.0493
Epoch: 059, MAE(train): 0.0525, MAE(val): 0.0639
Epoch: 069, MAE(train): 0.0496, MAE(val): 0.0485
Epoch: 079, MAE(train): 0.0495, MAE(val): 0.0514
Epoch: 089, MAE(train): 0.0478, MAE(val): 0.0472
Epoch: 099, MAE(train): 0.0460, MAE(val): 0.0419
Epoch: 109, MAE(train): 0.0460, MAE(val): 0.0547
Epoch: 119, MAE(train): 0.0455, MAE(val): 0.0523
Epoch: 129, MAE(train): 0.0440, MAE(val): 0.0459
Epoch: 139, MAE(train): 0.0439, MAE(val): 0.0570
Epoch: 149, MAE(train): 0.0427, MAE(val): 0.0453
Epoch: 159, MAE(train): 0.0431, MAE(val): 0.0416
Epoch: 169, MAE(train): 0.0419, MAE(val): 0.0397
Epoch: 179, MAE(train): 0.0419, MAE(val): 0.0448
Epoch: 189, MAE(train): 0.0409, MAE(val): 0.0498
Epoch: 199, MAE(train): 0.0403, MAE(val): 0.0559
Epoch: 209, MAE(trai

In [211]:
torch.save(obj=agnn2dq_model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f='/content/drive/MyDrive/2024_modeling/240811_aGNN2DQM.pth')